<h1 style="font-size: 50px;">Research Project 2 Epilepsy Ionic Modulation - SmallBrain Hippocampus Regions</h1>

This file contains the code for running a simulation where the model used is build by mimicking the regions present in the hippocampus: DG&CA3, CA1, Subiculum, and Enthorinal Cortex. For each region, the neuron densities and volumes were retrieved from literature and were used to generate ~17000 neurons across the entire 3D space.

<br></br><br></br>

<h2 style="font-size: 40px;">Installing and Importing Libraries</h2>

In [1]:
# Installing all the required libraries.
!pip install -q jupyter
!pip install -q matplotlib
!pip install -q numpy
!pip install -q pandas
!pip install -q plotly
!pip install -q scipy
!pip install -q tqdm
!pip install -q Brian2

In [2]:
# Importing all the required libraries.
from plots import *
from equations import *
from global_settings import *
from masks import *
from helper import *
from run_loop import *

In [3]:
import os
from brian2 import *
import random
from itertools import chain, zip_longest
from tqdm.notebook import tqdm
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import time
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

<br></br><br></br>

<h2 style="font-size: 40px;">General Functions</h2>

In [4]:
# Reads the stimulus input signal.
def read_input_signal(file_name):
    in_1 = np.loadtxt('./stimuli/'+file_name)

    # Converting the signal to be of type 'TimedArray' with a specified time step which can be used in the simulation.
    input_signal = TimedArray(in_1*namp,dt=defaultclock.dt)

    return input_signal

<br></br>

In [5]:
# Checking whether the mask with the provided coordinate center and radius can be applied to the list of regions.
def checking_mask(shape_mask, list_of_regions, center_coordinates, radius_or_edge_length, region_names, region_geometrics):

    # Checking if the 'list_of_regions' is empty.
    if len(list_of_regions) == 0:
        raise ValueError("The 'list_of_regions' list is empty.")
    
    # Checking if the 'list_of_regions' only contains valid region names.
    if not all(region in region_names for region in list_of_regions):
        raise ValueError("The 'list_of_regions' list contains an invalid region name.")
    
    # Checking if there are no duplicate region in the 'list_of_regions'.
    if len(list_of_regions) != len(set(list_of_regions)):
        raise ValueError("The 'list_of_regions' list contains duplicate regions.")

    # Checking that the 'list_of_regions' list only contains regions that are adjacent to each other (otherwise the stimulus mask would pass through regions that are not in this list).
    if len(list_of_regions) > 1:
        region_indices = [region_names.index(region) for region in list_of_regions]
        if not all(region_indices[i] + 1 == region_indices[i+1] for i in range(len(region_indices) - 1)):
            raise ValueError("The 'list_of_regions' list contains regions that are not adjacent to each other.")

    # Checking if the coordinates of the center of the mask are positive.
    if not (center_coordinates[0] > 0 and center_coordinates[1] > 0 and center_coordinates[2] > 0):
        raise ValueError("The coordinates of the center of the mask should be positive.")

    # Checking if the radius of the mask is positive.
    if not (radius_or_edge_length > 0):
        raise ValueError("The radius of the mask should be positive.")

    # Generating 'num_edge_points' random points that are located on the edge of the mask which is either spherical, cubical, or all.
    num_edge_points = 10000
    if shape_mask == "spherical":
        radius = radius_or_edge_length
        edge_points = generating_n_random_points_spherical_edge(num_edge_points, center_coordinates, radius)
        
    elif shape_mask == "cubical":
        edge_length = radius_or_edge_length
        edge_points = generating_n_random_points_cubical_edge(num_edge_points, center_coordinates, edge_length)
        
    else:
        raise ValueError("The shape of the mask given is not valid. Please select either 'spherical' or 'cubical'.")

    # Checking whether every point present in the 'edge_points' is located within the list of regions.
    for edge_point in edge_points:
        point_in_region = False
        for region in list_of_regions:
            match region:
                case "DG_CA3":
                    point_in_region = check_point_within_DG_CA3_region(edge_point, region_geometrics)
                case "CA1":
                    point_in_region = check_point_within_CA1_region(edge_point, region_geometrics)
                case "Sub":
                    point_in_region = check_point_within_Sub_region(edge_point, region_geometrics)
                case "EC":
                    point_in_region = check_point_within_EC_region(edge_point, region_geometrics)
            if point_in_region:
                break
        if not point_in_region:
            raise ValueError("Not every point present in the mask will be within the defined regions.")

    return [center_coordinates, radius_or_edge_length]

<br></br>

In [6]:
# Generating n random points that are located on the edge of the spherical mask.
def generating_n_random_points_spherical_edge(num_points, center_coordinates, radius):

    # Looping 'num_points' times and for each loop generating a random point on the edge of the spherical mask.
    points = []
    for i in range(num_points):
        # Generating the components needed to generate a point on the edge exactly 'radius' away from the center coordinates.
        phi = np.random.uniform(0, 2 * np.pi)
        costheta = np.random.uniform(-1, 1)
        theta = np.arccos(costheta)

        # Generating a random point that is located on the edge of the spherical mask.
        x = center_coordinates[0] + radius * np.sin(theta) * np.cos(phi)
        y = center_coordinates[1] + radius * np.sin(theta) * np.sin(phi)
        z = center_coordinates[2] + radius * np.cos(theta)

        # Appending the randomly generated point to the 'points' array.
        points.append([x, y, z])
        
    return points

<br></br>

In [7]:
# Generating n random points that are located on the edge of the cubical mask.
def generating_n_random_points_cubical_edge(num_points, center_coordinates, edge_length):

    # Looping 'num_points' times and for each loop generating a random point on the edge of the cubical mask.
    points = []
    for i in range(num_points):
        
        # Generating a random face of the cube on which the random point will be located.
        face = np.random.randint(0, 6)  

        # Depending on the face generated, a random point will be created.
        match face:
        
            # The right hand side face.
            case 0: 
                x = center_coordinates[0] - edge_length/2
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The left hand side face.
            case 1:
                x = center_coordinates[0] + edge_length/2
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The bottom face.
            case 2:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = center_coordinates[1] - edge_length/2
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The top face.
            case 3:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = center_coordinates[1] + edge_length/2
                z = np.random.uniform(center_coordinates[2] - edge_length/2, center_coordinates[2] + edge_length/2)

            # The front face.
            case 4:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = center_coordinates[2] - edge_length/2

            # The rear face.
            case 5:
                x = np.random.uniform(center_coordinates[0] - edge_length/2, center_coordinates[0] + edge_length/2)
                y = np.random.uniform(center_coordinates[1] - edge_length/2, center_coordinates[1] + edge_length/2)
                z = center_coordinates[2] + edge_length/2

        # Appending the randomly generated point to the 'points' array.
        points.append([x, y, z])

    return points

<br></br>

In [8]:
# Checking whether the point is located within the DG_CA3 region.
def check_point_within_DG_CA3_region(point, region_geometrics):

    # Retrieving the x-coordinate, y-coordinate, and z-coordinate from the 'point' parameter.
    x_point, y_point, z_point = point

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the DG_CA3 region from the 'region_geometrics' parameter.
    DG_CA3_bottom_left_x, DG_CA3_bottom_left_y, DG_CA3_bottom_left_z  = region_geometrics['DG_CA3_block']['bottom_left_position']

    # Retrieving the width, height, and width of the DG_CA3 region from the 'region_geometrics' parameter.
    DG_CA3_width = region_geometrics['DG_CA3_block']['width']
    DG_CA3_height = region_geometrics['DG_CA3_block']['height']
    DG_CA3_depth = region_geometrics['DG_CA3_block']['depth']

    # Checking whether the point is located within the DG_CA3 region.
    if ((x_point >= DG_CA3_bottom_left_x and x_point <= DG_CA3_bottom_left_x + DG_CA3_width) and
        (y_point >= DG_CA3_bottom_left_y and y_point <= DG_CA3_bottom_left_y + DG_CA3_height) and
        (z_point >= DG_CA3_bottom_left_z and z_point <= DG_CA3_bottom_left_z + DG_CA3_depth)):
        return True
    else:
        return False

<br></br>

In [9]:
# Checking whether the point is located within the CA1 region.
def check_point_within_CA1_region(point, region_geometrics):

    # Retrieving the x-coordinate, y-coordinate, and z-coordinate from the 'point' parameter.
    x_point, y_point, z_point = point

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the left block of the CA1 region from the 'region_geometrics' parameter.
    CA1_left_block_bottom_left_x, CA1_left_block_bottom_left_y, CA1_left_block_bottom_left_z  = region_geometrics['CA1_left_block']['bottom_left_position']

    # Retrieving the width, height, and width of the left block of the CA1 region from the 'region_geometrics' parameter.
    CA1_left_block_width = region_geometrics['CA1_left_block']['width']
    CA1_left_block_height = region_geometrics['CA1_left_block']['height']
    CA1_left_block_depth = region_geometrics['CA1_left_block']['depth']

    # Retrieving the bottom middle x-coordinate, y-coordinate, and z-coordinate of the middle block of the CA1 region from the 'region_geometrics' parameter.
    CA1_middle_block_bottom_left_x, CA1_middle_block_bottom_left_y, CA1_middle_block_bottom_left_z  = region_geometrics['CA1_middle_block']['bottom_left_position']

    # Retrieving the width, height, and width of the middle block of the CA1 region from the 'region_geometrics' parameter.
    CA1_middle_block_width = region_geometrics['CA1_middle_block']['width']
    CA1_middle_block_height = region_geometrics['CA1_middle_block']['height']
    CA1_middle_block_depth = region_geometrics['CA1_middle_block']['depth']

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the right block of the CA1 region from the 'region_geometrics' parameter.
    CA1_right_block_bottom_left_x, CA1_right_block_bottom_left_y, CA1_right_block_bottom_left_z  = region_geometrics['CA1_right_block']['bottom_left_position']

    # Retrieving the width, height, and width of the right block of the CA1 region from the 'region_geometrics' parameter.
    CA1_right_block_width = region_geometrics['CA1_right_block']['width']
    CA1_right_block_height = region_geometrics['CA1_right_block']['height']
    CA1_right_block_depth = region_geometrics['CA1_right_block']['depth']

    # Checking whether the point is located within the left block of the CA1 region.
    if ((x_point >= CA1_left_block_bottom_left_x and x_point <= CA1_left_block_bottom_left_x + CA1_left_block_width) and
        (y_point >= CA1_left_block_bottom_left_y and y_point <= CA1_left_block_bottom_left_y + CA1_left_block_height) and
        (z_point >= CA1_left_block_bottom_left_z and z_point <= CA1_left_block_bottom_left_z + CA1_left_block_depth)):
        return True
        
    # Checking whether the point is located within the middle block of the CA1 region.
    elif ((x_point >= CA1_middle_block_bottom_left_x and x_point <= CA1_middle_block_bottom_left_x + CA1_middle_block_width) and
        (y_point >= CA1_middle_block_bottom_left_y and y_point <= CA1_middle_block_bottom_left_y + CA1_middle_block_height) and
        (z_point >= CA1_middle_block_bottom_left_z and z_point <= CA1_middle_block_bottom_left_z + CA1_middle_block_depth)):
        return True
    
    # Checking whether the point is located within the right block of the CA1 region.
    elif ((x_point >= CA1_right_block_bottom_left_x and x_point <= CA1_right_block_bottom_left_x + CA1_right_block_width) and
        (y_point >= CA1_right_block_bottom_left_y and y_point <= CA1_right_block_bottom_left_y + CA1_right_block_height) and
        (z_point >= CA1_right_block_bottom_left_z and z_point <= CA1_right_block_bottom_left_z + CA1_right_block_depth)):
        return True
        
    else:
        return False

<br></br>

In [10]:
# Checking whether the point is located within the Sub region.
def check_point_within_Sub_region(point, region_geometrics):

    # Retrieving the x-coordinate, y-coordinate, and z-coordinate from the 'point' parameter.
    x_point, y_point, z_point = point

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the left block of the Sub region from the 'region_geometrics' parameter.
    Sub_left_block_bottom_left_x, Sub_left_block_bottom_left_y, Sub_left_block_bottom_left_z  = region_geometrics['Sub_left_block']['bottom_left_position']

    # Retrieving the width, height, and width of the left block of the Sub region from the 'region_geometrics' parameter.
    Sub_left_block_width = region_geometrics['Sub_left_block']['width']
    Sub_left_block_height = region_geometrics['Sub_left_block']['height']
    Sub_left_block_depth = region_geometrics['Sub_left_block']['depth']

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the right block of the Sub region from the 'region_geometrics' parameter.
    Sub_right_block_bottom_left_x, Sub_right_block_bottom_left_y, Sub_right_block_bottom_left_z  = region_geometrics['Sub_right_block']['bottom_left_position']

    # Retrieving the width, height, and width of the right block of the Sub region from the 'region_geometrics' parameter.
    Sub_right_block_width = region_geometrics['Sub_right_block']['width']
    Sub_right_block_height = region_geometrics['Sub_right_block']['height']
    Sub_right_block_depth = region_geometrics['Sub_right_block']['depth']

    # Checking whether the point is located within the left block of the Sub region.
    if ((x_point >= Sub_left_block_bottom_left_x and x_point <= Sub_left_block_bottom_left_x + Sub_left_block_width) and
        (y_point >= Sub_left_block_bottom_left_y and y_point <= Sub_left_block_bottom_left_y + Sub_left_block_height) and
        (z_point >= Sub_left_block_bottom_left_z and z_point <= Sub_left_block_bottom_left_z + Sub_left_block_depth)):
        return True
    
    # Checking whether the point is located within the right block of the Sub region.
    elif ((x_point >= Sub_right_block_bottom_left_x and x_point <= Sub_right_block_bottom_left_x + Sub_right_block_width) and
        (y_point >= Sub_right_block_bottom_left_y and y_point <= Sub_right_block_bottom_left_y + Sub_right_block_height) and
        (z_point >= Sub_right_block_bottom_left_z and z_point <= Sub_right_block_bottom_left_z + Sub_right_block_depth)):
        return True
        
    else:
        return False

<br></br>

In [11]:
# Checking whether the point is located within the EC region.
def check_point_within_EC_region(point, region_geometrics):

    # Retrieving the x-coordinate, y-coordinate, and z-coordinate from the 'point' parameter.
    x_point, y_point, z_point = point

    # Retrieving the bottom left x-coordinate, y-coordinate, and z-coordinate of the EC region from the 'region_geometrics' parameter.
    EC_bottom_left_x, EC_bottom_left_y, EC_bottom_left_z  = region_geometrics['EC_block']['bottom_left_position']

    # Retrieving the width, height, and width of the EC region from the 'region_geometrics' parameter.
    EC_width = region_geometrics['EC_block']['width']
    EC_height = region_geometrics['EC_block']['height']
    EC_depth = region_geometrics['EC_block']['depth']

    # Checking whether the point is located within the EC region.
    if ((x_point >= EC_bottom_left_x and x_point <= EC_bottom_left_x + EC_width) and
        (y_point >= EC_bottom_left_y and y_point <= EC_bottom_left_y + EC_height) and
        (z_point >= EC_bottom_left_z and z_point <= EC_bottom_left_z + EC_depth)):
        return True
    else:
        return False

<br></br><br></br>

<h2 style="font-size: 40px;">Topology and Neuron Groups Functions</h2>

In [12]:
# Creating the complete neuron topology of the model taking into account the structure of the hippocampus.
def create_complete_neuron_topology_hippocampus(desired_total_num_of_neurons, region_neuron_densities, region_volumes, region_excitatory_ratios, region_names):

    ############################################################
    ########  Calculating Number of Neurons per Region  ########
    ############################################################
    # Initializing a dictionary that will contain the number of neurons present in each of the regions and a variable that counts the total numbers of neurons.
    num_of_neurons = {}
    total_num_of_neurons = 0

    # Looping over every region and calculating how many neurons are present in each of the regions depending on the neuron densities and the volumes of the regions.
    for region in region_names:
        num_of_neurons[region] = region_neuron_densities[region] * region_volumes[region]
        total_num_of_neurons += num_of_neurons[region]

    # Calculating the scaling factor for the number of neurons for all regions as we ideally only want to model a number similar to the 'desired_total_num_of_neurons'.
    neuron_scaling_factor = desired_total_num_of_neurons / total_num_of_neurons

    # Initializing a dictionary that will contain the adjusted number of neurons present in each of the regions and one that will contain the adjusted volume of each region.
    num_of_neurons_adjusted = {}
    region_volume_adjusted = {}

    # Looping over every region and calculating how many neurons are present in each of the regions after having multiplied the original number with the 'neuron_scaling_factor'.
    for region in region_names:
        num_of_neurons_adjusted[region] = neuron_scaling_factor * num_of_neurons[region]
        region_volume_adjusted[region] = neuron_scaling_factor * region_volumes[region]

    # Creating a dictionary that will eventually contain all the region information.
    region_data = {}
    for region in region_names:
        region_data[region] = {"region_name": region,
                             "volume": region_volume_adjusted[region],
                             "num_excitatory_neurons": region_excitatory_ratios[region] * num_of_neurons_adjusted[region],
                             "num_inhibitory_neurons": (1 - region_excitatory_ratios[region]) * num_of_neurons_adjusted[region]}

    # Initializing a dictionary that will eventually contain the geometrical information of the different building blocks of the regions.
    region_geometrics = {}

    
    ############################################################
    ########  Calculating Dimensions of DG_CA3 and CA1  ########
    ############################################################
    # Calculating the dimensions of the DG_CA3 region as it located within the CA1 region. Here, we assume it is a cube with equal dimensions which can be retrieved by calculating the cube root of its volume.
    DG_CA3_dimensions = region_data['DG_CA3']['volume'] ** (1/3)

    # Calculating how many excitatory and inhibitory neurons are present in the DG_CA3 region.
    DG_CA3_block_volume = DG_CA3_dimensions * DG_CA3_dimensions * DG_CA3_dimensions
    DG_CA3_block_volume_prop = DG_CA3_block_volume / region_data['DG_CA3']['volume']
    DG_CA3_block_num_excitatory = int(region_data['DG_CA3']['num_excitatory_neurons'] * DG_CA3_block_volume_prop)
    DG_CA3_block_num_inhibitory = int(region_data['DG_CA3']['num_inhibitory_neurons'] * DG_CA3_block_volume_prop)

    # Calculating the total volume of the combined DG_CA3 and CA1 region by adding them together. 
    total_volume_DG_CA3_and_CA1 = region_data['DG_CA3']['volume'] + region_data['CA1']['volume']

    # As the height of the CA1 region is a bit higher we can multiply the height of the DG_CA3 region with 1.2 (determined through inspection).
    height_DG_CA3_and_CA1 = 1.2 * DG_CA3_dimensions

    # Setting the depth of the combined DG_CA3 and CA1 block to be equal to the depth of the DG_CA3 region alone.
    depth_DG_CA3_and_CA1 = DG_CA3_dimensions

    # Calculating the width of the combined DG_CA3 and CA1 block by dividing the total volume by the product of the height and depth of the combined DG_CA3 and CA1 block.
    width_DG_CA3_and_CA1 = total_volume_DG_CA3_and_CA1 / (height_DG_CA3_and_CA1 * depth_DG_CA3_and_CA1)

    ####################
    ## CA1 Left Block ##
    ####################
    # Calculating the width of the left block of the CA1 region by dividing the difference between the width of the DG_CA3 region and the width of the combined DG_CA3 and CA1 block by 2.
    CA1_left_block_width = (width_DG_CA3_and_CA1 - DG_CA3_dimensions) / 2

    # Setting the height and depth of the left block of the CA1 region to be equal to respectively the height and depth of the combined DG_CA3 and CA1 block.
    CA1_left_block_height = height_DG_CA3_and_CA1
    CA1_left_block_depth = depth_DG_CA3_and_CA1

    # Calculating how many excitatory and inhibitory neurons are present in the left block of the CA1 region.
    CA1_left_block_volume = CA1_left_block_width * CA1_left_block_height * CA1_left_block_depth
    CA1_left_block_volume_prop = CA1_left_block_volume / region_data['CA1']['volume']
    CA1_left_block_num_excitatory = int(region_data['CA1']['num_excitatory_neurons'] * CA1_left_block_volume_prop)
    CA1_left_block_num_inhibitory = int(region_data['CA1']['num_inhibitory_neurons'] * CA1_left_block_volume_prop)
    
    ######################
    ## CA1 Middle Block ##
    ######################
    # Calculating the height of the middle block of the CA1 region by subtracting the height of the DG_CA3 region from the height of the combined DG_CA3 and CA1 block.
    CA1_middle_block_height = height_DG_CA3_and_CA1 - DG_CA3_dimensions

    # Setting the width and depth of the middle block of the CA1 region to be equal to respectively the width of the DG_CA3 region and depth of the combined DG_CA3 and CA1 block.
    CA1_middle_block_width = DG_CA3_dimensions
    CA1_middle_block_depth = depth_DG_CA3_and_CA1

    # Calculating how many excitatory and inhibitory neurons are present in the middle block of the CA1 region.
    CA1_middle_block_volume = CA1_middle_block_width * CA1_middle_block_height * CA1_middle_block_depth
    CA1_middle_block_volume_prop = CA1_middle_block_volume / region_data['CA1']['volume']
    CA1_middle_block_num_excitatory = int(region_data['CA1']['num_excitatory_neurons'] * CA1_middle_block_volume_prop)
    CA1_middle_block_num_inhibitory = int(region_data['CA1']['num_inhibitory_neurons'] * CA1_middle_block_volume_prop)

    #####################
    ## CA1 Right Block ##
    #####################
    # Calculating the width of the right block of the CA1 region by dividing the difference between the width of the DG_CA3 region and the width of the combined DG_CA3 and CA1 block by 2.
    CA1_right_block_width = (width_DG_CA3_and_CA1 - DG_CA3_dimensions) / 2

    # Setting the height and depth of the right block of the CA1 region to be equal to respectively the height and depth of the combined DG_CA3 and CA1 block.
    CA1_right_block_height = height_DG_CA3_and_CA1
    CA1_right_block_depth = depth_DG_CA3_and_CA1

    # Calculating how many excitatory and inhibitory neurons are present in the right block of the CA1 region.
    CA1_right_block_volume = CA1_right_block_width * CA1_right_block_height * CA1_right_block_depth
    CA1_right_block_volume_prop = CA1_right_block_volume / region_data['CA1']['volume']
    CA1_right_block_num_excitatory = int(region_data['CA1']['num_excitatory_neurons'] * CA1_right_block_volume_prop)
    CA1_right_block_num_inhibitory = int(region_data['CA1']['num_inhibitory_neurons'] * CA1_right_block_volume_prop)

    # Adding the geometrical information of the three CA1 blocks to the 'region_geometrics' dictionary.
    CA1_left_block_x = 0
    CA1_left_block_y = 0
    CA1_left_block_z = 0
    region_geometrics['CA1_left_block'] = {"width": CA1_left_block_width,
                                         "height": CA1_left_block_height,
                                         "depth": CA1_left_block_depth,
                                         "bottom_left_position": [CA1_left_block_x, CA1_left_block_y, CA1_left_block_z],
                                         "num_excitatory_neurons": CA1_left_block_num_excitatory,
                                         "num_inhibitory_neurons": CA1_left_block_num_inhibitory
                                         }
    CA1_middle_block_x = CA1_left_block_x + CA1_left_block_width
    CA1_middle_block_y = CA1_left_block_y
    CA1_middle_block_z = 0
    region_geometrics['CA1_middle_block'] = {"width": CA1_middle_block_width,
                                           "height": CA1_middle_block_height,
                                           "depth": CA1_middle_block_depth,
                                           "bottom_left_position": [CA1_middle_block_x, CA1_middle_block_y, CA1_middle_block_z],
                                           "num_excitatory_neurons": CA1_middle_block_num_excitatory,
                                           "num_inhibitory_neurons": CA1_middle_block_num_inhibitory
                                           }
    CA1_right_block_x = CA1_middle_block_x + CA1_middle_block_width
    CA1_right_block_y = CA1_middle_block_y
    CA1_right_block_z = 0
    region_geometrics['CA1_right_block'] = {"width": CA1_right_block_width,
                                          "height": CA1_right_block_height,
                                          "depth": CA1_right_block_depth,
                                          "bottom_left_position": [CA1_right_block_x, CA1_right_block_y, CA1_right_block_z],
                                          "num_excitatory_neurons": CA1_right_block_num_excitatory,
                                          "num_inhibitory_neurons": CA1_right_block_num_inhibitory
                                          }

    # Adding the geometrical information of the DG_CA3 block to the 'region_geometrics' dictionary.
    DG_CA3_block_x = CA1_left_block_x + CA1_left_block_width
    DG_CA3_block_y = CA1_middle_block_y + CA1_middle_block_height
    DG_CA3_block_z = 0
    region_geometrics['DG_CA3_block'] = {"width": DG_CA3_dimensions,
                                       "height": DG_CA3_dimensions,
                                       "depth": DG_CA3_dimensions,
                                       "bottom_left_position": [DG_CA3_block_x, DG_CA3_block_y, DG_CA3_block_z],
                                       "num_excitatory_neurons": DG_CA3_block_num_excitatory,
                                       "num_inhibitory_neurons": DG_CA3_block_num_inhibitory
                                       }
    

    #################################################
    ########  Calculating Dimensions of Sub  ########
    #################################################
    ####################
    ## Sub Left Block ##
    ####################
    # Setting the width and depth of the left block of the Sub region to be equal to respectively the width and depth of the right block of the CA1 region.
    Sub_left_block_width = CA1_right_block_width
    Sub_left_block_depth = CA1_right_block_depth

    # Calculating the height of the left block of the Sub region by dividing half of the total volume of the Sub region by the product of the width and depth of the left block of the Sub region.
    Sub_left_block_height = (region_data['Sub']['volume'] / 2) / (Sub_left_block_width * Sub_left_block_depth)

    # Calculating how many excitatory and inhibitory neurons are present in the left block of the Sub region.
    Sub_left_block_volume = Sub_left_block_width * Sub_left_block_height * Sub_left_block_depth
    Sub_left_block_volume_prop = Sub_left_block_volume / region_data['Sub']['volume']
    Sub_left_block_num_excitatory = int(region_data['Sub']['num_excitatory_neurons'] * Sub_left_block_volume_prop)
    Sub_left_block_num_inhibitory = int(region_data['Sub']['num_inhibitory_neurons'] * Sub_left_block_volume_prop)

    #####################
    ## Sub Right Block ##
    #####################
    # Setting the width, height, and depth of the right block of the Sub region to be equal to respectively the height, width, and depth of the left block of the Sub region (the right block is a 90 degree transformation of the left block).
    Sub_right_block_width = Sub_left_block_height
    Sub_right_block_height = Sub_left_block_width
    Sub_right_block_depth = Sub_left_block_depth

    # Calculating how many excitatory and inhibitory neurons are present in the right block of the Sub region.
    Sub_right_block_volume = Sub_right_block_width * Sub_right_block_height * Sub_right_block_depth
    Sub_right_block_volume_prop = Sub_right_block_volume / region_data['Sub']['volume']
    Sub_right_block_num_excitatory = int(region_data['Sub']['num_excitatory_neurons'] * Sub_right_block_volume_prop)
    Sub_right_block_num_inhibitory = int(region_data['Sub']['num_inhibitory_neurons'] * Sub_right_block_volume_prop)

    # Adding the geometrical information of the two Sub blocks to the 'region_geometrics' dictionary.
    Sub_left_block_x = CA1_right_block_x
    Sub_left_block_y = CA1_right_block_y + CA1_right_block_height
    Sub_left_block_z = 0
    region_geometrics['Sub_left_block'] = {"width": Sub_left_block_width,
                                         "height": Sub_left_block_height,
                                         "depth": Sub_left_block_depth,
                                         "bottom_left_position": [Sub_left_block_x, Sub_left_block_y, Sub_left_block_z],
                                         "num_excitatory_neurons": Sub_left_block_num_excitatory,
                                         "num_inhibitory_neurons": Sub_left_block_num_inhibitory
                                         }
    Sub_right_block_x = Sub_left_block_x + Sub_left_block_width
    Sub_right_block_y = Sub_left_block_y + (Sub_left_block_height - Sub_right_block_height)
    Sub_right_block_z = 0
    region_geometrics['Sub_right_block'] = {"width": Sub_right_block_width,
                                          "height": Sub_right_block_height,
                                          "depth": Sub_right_block_depth,
                                          "bottom_left_position": [Sub_right_block_x, Sub_right_block_y, Sub_right_block_z],
                                          "num_excitatory_neurons": Sub_right_block_num_excitatory,
                                          "num_inhibitory_neurons": Sub_right_block_num_inhibitory
                                          }
    

    ################################################
    ########  Calculating Dimensions of EC  ########
    ################################################
    # Setting the width and depth of the EC region to be equal to respectively the width and depth of the left block of the Sub region.
    EC_width = Sub_left_block_width
    EC_depth = Sub_left_block_depth

    # Calculating the height of the EC region by dividing the volume of the EC region by the product of the width and depth of the EC region.
    EC_height = region_data['EC']['volume'] / (EC_width * EC_depth)

    # Calculating how many excitatory and inhibitory neurons are present in the EC region.
    EC_block_volume = EC_width * EC_height * EC_depth
    EC_block_volume_prop = EC_block_volume / region_data['EC']['volume']
    EC_block_num_excitatory = int(region_data['EC']['num_excitatory_neurons'] * EC_block_volume_prop)
    EC_block_num_inhibitory = int(region_data['EC']['num_inhibitory_neurons'] * EC_block_volume_prop)

    # Adding the geometrical information of the EC block to the 'region_geometrics' dictionary.
    EC_block_x = Sub_right_block_x + Sub_right_block_width
    EC_block_y = (Sub_right_block_y + Sub_right_block_height) - EC_height
    EC_block_z = 0
    region_geometrics['EC_block'] = {"width": EC_width,
                                   "height": EC_height,
                                   "depth": EC_depth,
                                   "bottom_left_position": [EC_block_x, EC_block_y, EC_block_z],
                                   "num_excitatory_neurons": EC_block_num_excitatory,
                                   "num_inhibitory_neurons": EC_block_num_inhibitory
                                   }

    
    ##########################################################
    ########  Updating Number of Neurons Region Data  ########
    ##########################################################
    # Updating the number of excitatory and inhibitory neurons for each of the regions based on the number of neurons calculated for each of the blocks.
    region_data['CA1']['num_excitatory_neurons'] = region_geometrics['CA1_left_block']['num_excitatory_neurons'] + region_geometrics['CA1_middle_block']['num_excitatory_neurons'] + region_geometrics['CA1_right_block']['num_excitatory_neurons']
    region_data['CA1']['num_inhibitory_neurons'] = region_geometrics['CA1_left_block']['num_inhibitory_neurons'] + region_geometrics['CA1_middle_block']['num_inhibitory_neurons'] + region_geometrics['CA1_right_block']['num_inhibitory_neurons']
    region_data['Sub']['num_excitatory_neurons'] = region_geometrics['Sub_left_block']['num_excitatory_neurons'] + region_geometrics['Sub_right_block']['num_excitatory_neurons']
    region_data['Sub']['num_inhibitory_neurons'] = region_geometrics['Sub_left_block']['num_inhibitory_neurons'] + region_geometrics['Sub_right_block']['num_inhibitory_neurons']
    region_data['DG_CA3']['num_excitatory_neurons'] = region_geometrics['DG_CA3_block']['num_excitatory_neurons']
    region_data['DG_CA3']['num_inhibitory_neurons'] = region_geometrics['DG_CA3_block']['num_inhibitory_neurons']
    region_data['EC']['num_excitatory_neurons'] = region_geometrics['EC_block']['num_excitatory_neurons']
    region_data['EC']['num_inhibitory_neurons'] = region_geometrics['EC_block']['num_inhibitory_neurons']

    
    #############################################################
    ########  Generating Random Points for every Region  ########
    #############################################################
    #########
    ## CA1 ##
    #########   
    # Generating a set of random x, y, and z positions that fall within the left block of the CA1 region for the excitatory neurons.
    CA1_left_block_x_exc = [random.uniform(region_geometrics['CA1_left_block']['bottom_left_position'][0], region_geometrics['CA1_left_block']['bottom_left_position'][0] + region_geometrics['CA1_left_block']['width']) for _ in range(region_geometrics['CA1_left_block']['num_excitatory_neurons'])]
    CA1_left_block_y_exc = [random.uniform(region_geometrics['CA1_left_block']['bottom_left_position'][1], region_geometrics['CA1_left_block']['bottom_left_position'][1] + region_geometrics['CA1_left_block']['height']) for _ in range(region_geometrics['CA1_left_block']['num_excitatory_neurons'])]
    CA1_left_block_z_exc = [random.uniform(region_geometrics['CA1_left_block']['bottom_left_position'][2], region_geometrics['CA1_left_block']['bottom_left_position'][2] + region_geometrics['CA1_left_block']['depth']) for _ in range(region_geometrics['CA1_left_block']['num_excitatory_neurons'])]
    CA1_left_block_excitatory_topology = np.array([CA1_left_block_x_exc, CA1_left_block_y_exc, CA1_left_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the left block of the CA1 region for the inhibitory neurons.
    CA1_left_block_x_inh = [random.uniform(region_geometrics['CA1_left_block']['bottom_left_position'][0], region_geometrics['CA1_left_block']['bottom_left_position'][0] + region_geometrics['CA1_left_block']['width']) for _ in range(region_geometrics['CA1_left_block']['num_inhibitory_neurons'])]
    CA1_left_block_y_inh = [random.uniform(region_geometrics['CA1_left_block']['bottom_left_position'][1], region_geometrics['CA1_left_block']['bottom_left_position'][1] + region_geometrics['CA1_left_block']['height']) for _ in range(region_geometrics['CA1_left_block']['num_inhibitory_neurons'])]
    CA1_left_block_z_inh = [random.uniform(region_geometrics['CA1_left_block']['bottom_left_position'][2], region_geometrics['CA1_left_block']['bottom_left_position'][2] + region_geometrics['CA1_left_block']['depth']) for _ in range(region_geometrics['CA1_left_block']['num_inhibitory_neurons'])]
    CA1_left_block_inhibitory_topology = np.array([CA1_left_block_x_inh, CA1_left_block_y_inh, CA1_left_block_z_inh])

    # Generating a set of random x, y, and z positions that fall within the middle block of the CA1 region for the excitatory neurons.
    CA1_middle_block_x_exc = [random.uniform(region_geometrics['CA1_middle_block']['bottom_left_position'][0], region_geometrics['CA1_middle_block']['bottom_left_position'][0] + region_geometrics['CA1_middle_block']['width']) for _ in range(region_geometrics['CA1_middle_block']['num_excitatory_neurons'])]
    CA1_middle_block_y_exc = [random.uniform(region_geometrics['CA1_middle_block']['bottom_left_position'][1], region_geometrics['CA1_middle_block']['bottom_left_position'][1] + region_geometrics['CA1_middle_block']['height']) for _ in range(region_geometrics['CA1_middle_block']['num_excitatory_neurons'])]
    CA1_middle_block_z_exc = [random.uniform(region_geometrics['CA1_middle_block']['bottom_left_position'][2], region_geometrics['CA1_middle_block']['bottom_left_position'][2] + region_geometrics['CA1_middle_block']['depth']) for _ in range(region_geometrics['CA1_middle_block']['num_excitatory_neurons'])]
    CA1_middle_block_excitatory_topology = np.array([CA1_middle_block_x_exc, CA1_middle_block_y_exc, CA1_middle_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the middle block of the CA1 region for the inhibitory neurons.
    CA1_middle_block_x_inh = [random.uniform(region_geometrics['CA1_middle_block']['bottom_left_position'][0], region_geometrics['CA1_middle_block']['bottom_left_position'][0] + region_geometrics['CA1_middle_block']['width']) for _ in range(region_geometrics['CA1_middle_block']['num_inhibitory_neurons'])]
    CA1_middle_block_y_inh = [random.uniform(region_geometrics['CA1_middle_block']['bottom_left_position'][1], region_geometrics['CA1_middle_block']['bottom_left_position'][1] + region_geometrics['CA1_middle_block']['height']) for _ in range(region_geometrics['CA1_middle_block']['num_inhibitory_neurons'])]
    CA1_middle_block_z_inh = [random.uniform(region_geometrics['CA1_middle_block']['bottom_left_position'][2], region_geometrics['CA1_middle_block']['bottom_left_position'][2] + region_geometrics['CA1_middle_block']['depth']) for _ in range(region_geometrics['CA1_middle_block']['num_inhibitory_neurons'])]
    CA1_middle_block_inhibitory_topology = np.array([CA1_middle_block_x_inh, CA1_middle_block_y_inh, CA1_middle_block_z_inh])

    # Generating a set of random x, y, and z positions that fall within the right block of the CA1 region for the excitatory neurons.
    CA1_right_block_x_exc = [random.uniform(region_geometrics['CA1_right_block']['bottom_left_position'][0], region_geometrics['CA1_right_block']['bottom_left_position'][0] + region_geometrics['CA1_right_block']['width']) for _ in range(region_geometrics['CA1_right_block']['num_excitatory_neurons'])]
    CA1_right_block_y_exc = [random.uniform(region_geometrics['CA1_right_block']['bottom_left_position'][1], region_geometrics['CA1_right_block']['bottom_left_position'][1] + region_geometrics['CA1_right_block']['height']) for _ in range(region_geometrics['CA1_right_block']['num_excitatory_neurons'])]
    CA1_right_block_z_exc = [random.uniform(region_geometrics['CA1_right_block']['bottom_left_position'][2], region_geometrics['CA1_right_block']['bottom_left_position'][2] + region_geometrics['CA1_right_block']['depth']) for _ in range(region_geometrics['CA1_right_block']['num_excitatory_neurons'])]
    CA1_right_block_excitatory_topology = np.array([CA1_right_block_x_exc, CA1_right_block_y_exc, CA1_right_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the right block of the CA1 region for the inhibitory neurons.
    CA1_right_block_x_inh = [random.uniform(region_geometrics['CA1_right_block']['bottom_left_position'][0], region_geometrics['CA1_right_block']['bottom_left_position'][0] + region_geometrics['CA1_right_block']['width']) for _ in range(region_geometrics['CA1_right_block']['num_inhibitory_neurons'])]
    CA1_right_block_y_inh = [random.uniform(region_geometrics['CA1_right_block']['bottom_left_position'][1], region_geometrics['CA1_right_block']['bottom_left_position'][1] + region_geometrics['CA1_right_block']['height']) for _ in range(region_geometrics['CA1_right_block']['num_inhibitory_neurons'])]
    CA1_right_block_z_inh = [random.uniform(region_geometrics['CA1_right_block']['bottom_left_position'][2], region_geometrics['CA1_right_block']['bottom_left_position'][2] + region_geometrics['CA1_right_block']['depth']) for _ in range(region_geometrics['CA1_right_block']['num_inhibitory_neurons'])]
    CA1_right_block_inhibitory_topology = np.array([CA1_right_block_x_inh, CA1_right_block_y_inh, CA1_right_block_z_inh])
    
    #########
    ## Sub ##
    #########
    # Generating a set of random x, y, and z positions that fall within the left block of the Sub region for the excitatory neurons.
    Sub_left_block_x_exc = [random.uniform(region_geometrics['Sub_left_block']['bottom_left_position'][0], region_geometrics['Sub_left_block']['bottom_left_position'][0] + region_geometrics['Sub_left_block']['width']) for _ in range(region_geometrics['Sub_left_block']['num_excitatory_neurons'])]
    Sub_left_block_y_exc = [random.uniform(region_geometrics['Sub_left_block']['bottom_left_position'][1], region_geometrics['Sub_left_block']['bottom_left_position'][1] + region_geometrics['Sub_left_block']['height']) for _ in range(region_geometrics['Sub_left_block']['num_excitatory_neurons'])]
    Sub_left_block_z_exc = [random.uniform(region_geometrics['Sub_left_block']['bottom_left_position'][2], region_geometrics['Sub_left_block']['bottom_left_position'][2] + region_geometrics['Sub_left_block']['depth']) for _ in range(region_geometrics['Sub_left_block']['num_excitatory_neurons'])]
    Sub_left_block_excitatory_topology = np.array([Sub_left_block_x_exc, Sub_left_block_y_exc, Sub_left_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the left block of the Sub region for the inhibitory neurons.
    Sub_left_block_x_inh = [random.uniform(region_geometrics['Sub_left_block']['bottom_left_position'][0], region_geometrics['Sub_left_block']['bottom_left_position'][0] + region_geometrics['Sub_left_block']['width']) for _ in range(region_geometrics['Sub_left_block']['num_inhibitory_neurons'])]
    Sub_left_block_y_inh = [random.uniform(region_geometrics['Sub_left_block']['bottom_left_position'][1], region_geometrics['Sub_left_block']['bottom_left_position'][1] + region_geometrics['Sub_left_block']['height']) for _ in range(region_geometrics['Sub_left_block']['num_inhibitory_neurons'])]
    Sub_left_block_z_inh = [random.uniform(region_geometrics['Sub_left_block']['bottom_left_position'][2], region_geometrics['Sub_left_block']['bottom_left_position'][2] + region_geometrics['Sub_left_block']['depth']) for _ in range(region_geometrics['Sub_left_block']['num_inhibitory_neurons'])]
    Sub_left_block_inhibitory_topology = np.array([Sub_left_block_x_inh, Sub_left_block_y_inh, Sub_left_block_z_inh])

    # Generating a set of random x, y, and z positions that fall within the right block of the Sub region for the excitatory neurons.
    Sub_right_block_x_exc = [random.uniform(region_geometrics['Sub_right_block']['bottom_left_position'][0], region_geometrics['Sub_right_block']['bottom_left_position'][0] + region_geometrics['Sub_right_block']['width']) for _ in range(region_geometrics['Sub_right_block']['num_excitatory_neurons'])]
    Sub_right_block_y_exc = [random.uniform(region_geometrics['Sub_right_block']['bottom_left_position'][1], region_geometrics['Sub_right_block']['bottom_left_position'][1] + region_geometrics['Sub_right_block']['height']) for _ in range(region_geometrics['Sub_right_block']['num_excitatory_neurons'])]
    Sub_right_block_z_exc = [random.uniform(region_geometrics['Sub_right_block']['bottom_left_position'][2], region_geometrics['Sub_right_block']['bottom_left_position'][2] + region_geometrics['Sub_right_block']['depth']) for _ in range(region_geometrics['Sub_right_block']['num_excitatory_neurons'])]
    Sub_right_block_excitatory_topology = np.array([Sub_right_block_x_exc, Sub_right_block_y_exc, Sub_right_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the right block of the Sub region for the inhibitory neurons.
    Sub_right_block_x_inh = [random.uniform(region_geometrics['Sub_right_block']['bottom_left_position'][0], region_geometrics['Sub_right_block']['bottom_left_position'][0] + region_geometrics['Sub_right_block']['width']) for _ in range(region_geometrics['Sub_right_block']['num_inhibitory_neurons'])]
    Sub_right_block_y_inh = [random.uniform(region_geometrics['Sub_right_block']['bottom_left_position'][1], region_geometrics['Sub_right_block']['bottom_left_position'][1] + region_geometrics['Sub_right_block']['height']) for _ in range(region_geometrics['Sub_right_block']['num_inhibitory_neurons'])]
    Sub_right_block_z_inh = [random.uniform(region_geometrics['Sub_right_block']['bottom_left_position'][2], region_geometrics['Sub_right_block']['bottom_left_position'][2] + region_geometrics['Sub_right_block']['depth']) for _ in range(region_geometrics['Sub_right_block']['num_inhibitory_neurons'])]
    Sub_right_block_inhibitory_topology = np.array([Sub_right_block_x_inh, Sub_right_block_y_inh, Sub_right_block_z_inh])

    ############
    ## DG_CA3 ##
    ############
    # Generating a set of random x, y, and z positions that fall within the left block of the DG_CA3 region for the excitatory neurons.
    DG_CA3_block_x_exc = [random.uniform(region_geometrics['DG_CA3_block']['bottom_left_position'][0], region_geometrics['DG_CA3_block']['bottom_left_position'][0] + region_geometrics['DG_CA3_block']['width']) for _ in range(region_geometrics['DG_CA3_block']['num_excitatory_neurons'])]
    DG_CA3_block_y_exc = [random.uniform(region_geometrics['DG_CA3_block']['bottom_left_position'][1], region_geometrics['DG_CA3_block']['bottom_left_position'][1] + region_geometrics['DG_CA3_block']['height']) for _ in range(region_geometrics['DG_CA3_block']['num_excitatory_neurons'])]
    DG_CA3_block_z_exc = [random.uniform(region_geometrics['DG_CA3_block']['bottom_left_position'][2], region_geometrics['DG_CA3_block']['bottom_left_position'][2] + region_geometrics['DG_CA3_block']['depth']) for _ in range(region_geometrics['DG_CA3_block']['num_excitatory_neurons'])]
    DG_CA3_block_excitatory_topology = np.array([DG_CA3_block_x_exc, DG_CA3_block_y_exc, DG_CA3_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the left block of the DG_CA3 region for the inhibitory neurons.
    DG_CA3_block_x_inh = [random.uniform(region_geometrics['DG_CA3_block']['bottom_left_position'][0], region_geometrics['DG_CA3_block']['bottom_left_position'][0] + region_geometrics['DG_CA3_block']['width']) for _ in range(region_geometrics['DG_CA3_block']['num_inhibitory_neurons'])]
    DG_CA3_block_y_inh = [random.uniform(region_geometrics['DG_CA3_block']['bottom_left_position'][1], region_geometrics['DG_CA3_block']['bottom_left_position'][1] + region_geometrics['DG_CA3_block']['height']) for _ in range(region_geometrics['DG_CA3_block']['num_inhibitory_neurons'])]
    DG_CA3_block_z_inh = [random.uniform(region_geometrics['DG_CA3_block']['bottom_left_position'][2], region_geometrics['DG_CA3_block']['bottom_left_position'][2] + region_geometrics['DG_CA3_block']['depth']) for _ in range(region_geometrics['DG_CA3_block']['num_inhibitory_neurons'])]
    DG_CA3_block_inhibitory_topology = np.array([DG_CA3_block_x_inh, DG_CA3_block_y_inh, DG_CA3_block_z_inh])

    ########
    ## EC ##
    ########
    # Generating a set of random x, y, and z positions that fall within the left block of the EC region for the excitatory neurons.
    EC_block_x_exc = [random.uniform(region_geometrics['EC_block']['bottom_left_position'][0], region_geometrics['EC_block']['bottom_left_position'][0] + region_geometrics['EC_block']['width']) for _ in range(region_geometrics['EC_block']['num_excitatory_neurons'])]
    EC_block_y_exc = [random.uniform(region_geometrics['EC_block']['bottom_left_position'][1], region_geometrics['EC_block']['bottom_left_position'][1] + region_geometrics['EC_block']['height']) for _ in range(region_geometrics['EC_block']['num_excitatory_neurons'])]
    EC_block_z_exc = [random.uniform(region_geometrics['EC_block']['bottom_left_position'][2], region_geometrics['EC_block']['bottom_left_position'][2] + region_geometrics['EC_block']['depth']) for _ in range(region_geometrics['EC_block']['num_excitatory_neurons'])]
    EC_block_excitatory_topology = np.array([EC_block_x_exc, EC_block_y_exc, EC_block_z_exc])

    # Generating a set of random x, y, and z positions that fall within the left block of the EC region for the inhibitory neurons.
    EC_block_x_inh = [random.uniform(region_geometrics['EC_block']['bottom_left_position'][0], region_geometrics['EC_block']['bottom_left_position'][0] + region_geometrics['EC_block']['width']) for _ in range(region_geometrics['EC_block']['num_inhibitory_neurons'])]
    EC_block_y_inh = [random.uniform(region_geometrics['EC_block']['bottom_left_position'][1], region_geometrics['EC_block']['bottom_left_position'][1] + region_geometrics['EC_block']['height']) for _ in range(region_geometrics['EC_block']['num_inhibitory_neurons'])]
    EC_block_z_inh = [random.uniform(region_geometrics['EC_block']['bottom_left_position'][2], region_geometrics['EC_block']['bottom_left_position'][2] + region_geometrics['EC_block']['depth']) for _ in range(region_geometrics['EC_block']['num_inhibitory_neurons'])]
    EC_block_inhibitory_topology = np.array([EC_block_x_inh, EC_block_y_inh, EC_block_z_inh])

    
    ###################################################
    ########  Combining Topologies per Region  ########
    ###################################################
    # Initializing two dictionaries that will for each region store the positions of the excitatory and inhibitory neurons.
    excitatory_positions = {}
    inhibitory_positions = {}

    # Adding the excitatory and inhibitory topologies of the CA1 region to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['CA1'] = []
    for sublist1, sublist2, sublist3 in zip_longest(CA1_left_block_excitatory_topology, CA1_middle_block_excitatory_topology, CA1_right_block_excitatory_topology, fillvalue=[]):
        merged_sublist = list(chain(sublist1, sublist2, sublist3))
        excitatory_positions['CA1'].append(merged_sublist)
    inhibitory_positions['CA1'] = []
    for sublist1, sublist2, sublist3 in zip_longest(CA1_left_block_inhibitory_topology, CA1_middle_block_inhibitory_topology, CA1_right_block_inhibitory_topology, fillvalue=[]):
        merged_sublist = list(chain(sublist1, sublist2, sublist3))
        inhibitory_positions['CA1'].append(merged_sublist)
        
    # Adding the excitatory and inhibitory topologies of the Sub region to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['Sub'] = []
    for sublist1, sublist2 in zip_longest(Sub_left_block_excitatory_topology, Sub_right_block_excitatory_topology, fillvalue=[]):
        merged_sublist = list(chain(sublist1, sublist2))
        excitatory_positions['Sub'].append(merged_sublist)
    inhibitory_positions['Sub'] = []
    for sublist1, sublist2 in zip_longest(Sub_left_block_inhibitory_topology, Sub_right_block_inhibitory_topology, fillvalue=[]):
        merged_sublist = list(chain(sublist1, sublist2))
        inhibitory_positions['Sub'].append(merged_sublist)

    # Adding the excitatory and inhibitory topologies of the DG_CA3 region to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['DG_CA3'] = DG_CA3_block_excitatory_topology
    inhibitory_positions['DG_CA3'] = DG_CA3_block_inhibitory_topology

    # Adding the excitatory and inhibitory topologies of the EC region to the 'excitatory_positions' and 'inhibitory_positions' dictionaries.
    excitatory_positions['EC'] = EC_block_excitatory_topology
    inhibitory_positions['EC'] = EC_block_inhibitory_topology

    
    return region_data, region_geometrics, excitatory_positions, inhibitory_positions

<br></br>

In [13]:
# Creates the group of excitatory neurons.
def create_group_py(topology, noise, masks, current_region, stimulus_regions, treatment_regions, excitatory_topologies_lengths, group_name='exc_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise
    stimulus_mask, treatment_mask = masks

    # Adjusting the group name.
    group_name = group_name + "_" + current_region

    # Initializing a group of excitatory neurons with the following parameters:
    # - py_eqs => Differential equations that define the behavior of the excitatory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - reset => Resets neuron states after they fire according to 'reset_eqs'.
    # - refractory => Sets a refractory period during which an excitatory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_exc = NeuronGroup(len(x),py_eqs,threshold='v>V_th',reset=reset_eqs,refractory=3*ms,name=group_name, method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -100 mV.
    G_exc.v = '-60*mvolt-rand()*40*mvolt'

    # Sets the neurotransmitter to be used to be glutamate (which is an excitatory neurotransmitter).
    G_exc.glu = 1

    # Assigns the positions of the neurons in micrometers.
    G_exc.x = x * um
    G_exc.y = y * um
    G_exc.z = z * um

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_exc.taille = taille_exc_normale

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_exc.mu_noise = mu_noise
    G_exc.sigma_noise = sigma_noise

    # Applying the treatment mask to the neuron by first checking whether the current region is in the list of treatment regions. If not, a list of zeros is added as treatment mask.
    if current_region in treatment_regions:
        if current_region == 'DG_CA3':
            length_index1 = len(x)
            G_exc.treatment_mask = treatment_mask[:length_index1]
        elif current_region == 'CA1':
            if 'DG_CA3' in treatment_regions:
                length_index0 = excitatory_topologies_lengths[0]
                length_index1 = length_index0 + len(x)
            else:
                length_index0 = 0
                length_index1 = len(x)
            G_exc.treatment_mask = treatment_mask[length_index0:length_index1]
        elif current_region == 'Sub':
            if 'DG_CA3' in treatment_regions:
                if 'CA1' in treatment_regions:
                    length_index0 = excitatory_topologies_lengths[0] + excitatory_topologies_lengths[1]
                    length_index1 = length_index0 + len(x)
                else:
                    length_index0 = excitatory_topologies_lengths[0]
                    length_index1 = length_index0 + len(x)
            elif 'CA1' in treatment_regions:
                length_index0 = excitatory_topologies_lengths[1]
                length_index1 = length_index0 + len(x)
            else:
                length_index0 = 0
                length_index1 = len(x)
            G_exc.treatment_mask = treatment_mask[length_index0:length_index1]
        elif current_region == 'EC':
            length_index0 = -len(x)
            G_exc.treatment_mask = treatment_mask[length_index0:]
    else:
        G_exc.treatment_mask = np.zeros(len(x))

    # Applying the stimulus mask to the neuron by first checking whether the current region is in the list of stimulus regions. If not, a list of zeros is added as stimulus mask.
    if current_region in stimulus_regions:
        if current_region == 'DG_CA3':
            G_exc.stimulus_mask = stimulus_mask[:len(x)]
        else:
            G_exc.stimulus_mask = stimulus_mask[-len(x):]
    else:
        G_exc.stimulus_mask = np.zeros(len(x))
        
    return G_exc

<br></br>

In [14]:
# Creates the group of inhibitory neurons.
def create_group_inh(topology, noise, treatment_mask, current_region, treatment_regions, inhibitory_topologies_lengths, group_name='inh_group', integ_method='exponential_euler'):

    # Extracting the passed on parameters.
    x, y, z = topology
    mu_noise, sigma_noise = noise

    # Adjusting the group name.
    group_name = group_name + "_" + current_region

    # Initializing a group of inhibitory neurons with the following parameters:
    # - inh_eqs => Differential equations that define the behavior of the inhibitory neuron.
    # - threshold => Neurons fire an action potential when their membrane potential 'v' exceeds the threshold 'V_th'.
    # - refractory => Sets a refractory period during which an inhibitory neuron cannot fire again.
    # - method => Specifies the integration method for solving the differential equations.
    G_inh = NeuronGroup(len(x),inh_eqs,threshold='v>V_th', name=group_name, refractory=3*ms,method=integ_method)

    # Initializing the membrane potential 'v' to be a random value between -60 mV and -70 mV.
    G_inh.v = -60*mvolt-rand()*10*mvolt

    # Sets the size of the neurons to 'taille_exc_normale'.
    G_inh.taille = taille_inh_normale

    # Assigns the positions of the neurons in micrometers.
    G_inh.x = x * um
    G_inh.y = y * um
    G_inh.z = z * um

    # Sets the mean and standard deviation of the noise affecting the neurons.
    G_inh.mu_noise = mu_noise
    G_inh.sigma_noise = sigma_noise

    # Applying the treatment mask to the neuron by first checking whether the current region is in the list of treatment regions. If not, a list of zeros is added as treatment mask.
    if current_region in treatment_regions:
        if current_region == 'DG_CA3':
            length_index1 = len(x)
            G_inh.treatment_mask = treatment_mask[:length_index1]
        elif current_region == 'CA1':
            if 'DG_CA3' in treatment_regions:
                length_index0 = inhibitory_topologies_lengths[0]
                length_index1 = length_index0 + len(x)
            else:
                length_index0 = 0
                length_index1 = len(x)
            G_inh.treatment_mask = treatment_mask[length_index0:length_index1]
        elif current_region == 'Sub':
            if 'DG_CA3' in treatment_regions:
                if 'CA1' in treatment_regions:
                    length_index0 = inhibitory_topologies_lengths[0] + inhibitory_topologies_lengths[1]
                    length_index1 = length_index0 + len(x)
                else:
                    length_index0 = inhibitory_topologies_lengths[0]
                    length_index1 = length_index0 + len(x)
            elif 'CA1' in treatment_regions:
                length_index0 = inhibitory_topologies_lengths[1]
                length_index1 = length_index0 + len(x)
            else:
                length_index0 = 0
                length_index1 = len(x)
            G_inh.treatment_mask = treatment_mask[length_index0:length_index1]
        elif current_region == 'EC':
            length_index0 = -len(x)
            G_inh.treatment_mask = treatment_mask[length_index0:]
    else:
        G_inh.treatment_mask = np.zeros(len(x))

    return G_inh

<br></br>

In [15]:
# Creates a group for Local Field Potential (LFP) recording.
def create_group_lfp(LFP_electrode_region, region_geometrics):
    
    # Setting up a singular LFP electrode
    Ne = 1

    # Setting the resistivity of the extracellular field to 0.3 siemens per meter, which is within the typical range for biological tissue (0.3-0.4 S/m).
    sigma = 0.3*siemens/meter

    # Initializes a group of neurons, which will only consist of 1 representing a single LFP electrode which has three state variables:
    # - v : volt => Represents the voltage (LFP signal).
    # - x : meter => Represent the x-coordinate of the electrode.
    # - y : meter => Represent the y-coordinate of the electrode.
    # - z : meter => Represent the z-coordinate of the electrode.
    lfp = NeuronGroup(Ne, model='''v : volt
                                   x : meter
                                   y : meter
                                   z : meter''')

    # Defining the x, y, and z coordinates of the LFP electrode based on what the region of the LFP electrode is.
    if LFP_electrode_region == "DG_CA3":
        lfp.x = (region_geometrics['DG_CA3_block']['bottom_left_position'][0] + (region_geometrics['DG_CA3_block']['width'] / 2)) * mm
        lfp.y = (region_geometrics['DG_CA3_block']['bottom_left_position'][1] + (region_geometrics['DG_CA3_block']['height'] / 2)) * mm
        lfp.z = (region_geometrics['DG_CA3_block']['bottom_left_position'][2] + (region_geometrics['DG_CA3_block']['depth'] / 2)) * mm
    if LFP_electrode_region == "CA1":
        lfp.x = (region_geometrics['CA1_middle_block']['bottom_left_position'][0] + (region_geometrics['CA1_middle_block']['width'] / 2)) * mm
        lfp.y = (region_geometrics['CA1_middle_block']['bottom_left_position'][1] + (region_geometrics['CA1_middle_block']['height'] / 2)) * mm
        lfp.z = (region_geometrics['CA1_middle_block']['bottom_left_position'][2] + (region_geometrics['CA1_middle_block']['depth'] / 2)) * mm
    if LFP_electrode_region == "Sub":
        lfp.x = (region_geometrics['Sub_left_block']['bottom_left_position'][0] + (region_geometrics['Sub_left_block']['width'] / 2)) * mm
        lfp.y = (region_geometrics['Sub_left_block']['bottom_left_position'][1] + (region_geometrics['Sub_left_block']['height'] / 2)) * mm
        lfp.z = (region_geometrics['Sub_left_block']['bottom_left_position'][2] + (region_geometrics['Sub_left_block']['depth'] / 2)) * mm
    if LFP_electrode_region == "EC":
        lfp.x = (region_geometrics['EC_block']['bottom_left_position'][0] + (region_geometrics['EC_block']['width'] / 2)) * mm
        lfp.y = (region_geometrics['EC_block']['bottom_left_position'][1] + (region_geometrics['EC_block']['height'] / 2)) * mm
        lfp.z = (region_geometrics['EC_block']['bottom_left_position'][2] + (region_geometrics['EC_block']['depth'] / 2)) * mm

    return lfp

<br></br><br></br>

<h2 style="font-size: 40px;">Network Configuration Function</h2>

In [16]:
# Prepares the network of neurons. The topology is already there, but the neurons still need to be connected, which is done here.
def prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables):

    # Extracting the connection probabilities between neurons from the same region and from different regions.
    probabilities_between_regions = current_variables['probabilities_between_regions']
    probabilities_within_regions = current_variables['probabilities_within_regions']

    # Extracting the topologies from the passed on parameter 'topologies'.
    topology_DG_CA3_exc, topology_CA1_exc, topology_Sub_exc, topology_EC_exc, topology_DG_CA3_inh, topology_CA1_inh, topology_Sub_inh, topology_EC_inh = topologies

    # Extracting the treatment masks from the passed on parameter 'treatment_masks'.
    treatment_mask_exc, treatment_mask_inh = treatment_masks

    # Calculating the lengths of the excitatory and inhibitory topologies.
    excitatory_topologies_lengths = [len(topology_DG_CA3_exc[0]), len(topology_CA1_exc[0]), len(topology_Sub_exc[0]), len(topology_EC_exc[0])]
    inhibitory_topologies_lengths = [len(topology_DG_CA3_inh[0]), len(topology_CA1_inh[0]), len(topology_Sub_inh[0]), len(topology_EC_inh[0])]

    
    ##########################################
    ########  Creating Neuron Groups  ########
    ##########################################
    # Creating excitatory neuron groups for each region.
    G_DG_CA3_exc = create_group_py(topology_DG_CA3_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'DG_CA3', current_variables['stimulus_regions'], current_variables['treatment_regions'], excitatory_topologies_lengths)
    G_CA1_exc = create_group_py(topology_CA1_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'CA1', current_variables['stimulus_regions'], current_variables['treatment_regions'], excitatory_topologies_lengths)
    G_Sub_exc = create_group_py(topology_Sub_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'Sub', current_variables['stimulus_regions'], current_variables['treatment_regions'], excitatory_topologies_lengths)
    G_EC_exc = create_group_py(topology_EC_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc], 'EC', current_variables['stimulus_regions'], current_variables['treatment_regions'], excitatory_topologies_lengths)

    # Creating inhibitory neuron groups for each region.
    G_DG_CA3_inh = create_group_inh(topology_DG_CA3_inh, current_variables['noise_inh'], treatment_mask_inh, 'DG_CA3', current_variables['treatment_regions'], inhibitory_topologies_lengths)
    G_CA1_inh = create_group_inh(topology_CA1_inh, current_variables['noise_inh'], treatment_mask_inh, 'CA1', current_variables['treatment_regions'], inhibitory_topologies_lengths)
    G_Sub_inh = create_group_inh(topology_Sub_inh, current_variables['noise_inh'], treatment_mask_inh, 'Sub', current_variables['treatment_regions'], inhibitory_topologies_lengths)
    G_EC_inh = create_group_inh(topology_EC_inh, current_variables['noise_inh'], treatment_mask_inh, 'EC', current_variables['treatment_regions'], inhibitory_topologies_lengths)

    # Setting up a Local Field Potential (LFP) electrode.
    G_lfp = create_group_lfp(current_variables['LFP_electrode_region'], current_variables['region_geometrics'])
    
    # Adding the neuron groups to a list.
    neuron_groups = [G_DG_CA3_exc, G_CA1_exc, G_Sub_exc, G_EC_exc, G_DG_CA3_inh, G_CA1_inh, G_Sub_inh, G_EC_inh, G_lfp]

    
    #####################################
    ########  Creating Monitors  ########
    #####################################
    # Setting up monitors that monitor the population firing rate of all excitatory and inhibitory neuron groups.
    popmon_DG_CA3_exc = PopulationRateMonitor(G_DG_CA3_exc)
    popmon_CA1_exc = PopulationRateMonitor(G_CA1_exc)
    popmon_Sub_exc = PopulationRateMonitor(G_Sub_exc)
    popmon_EC_exc = PopulationRateMonitor(G_EC_exc)
    popmon_DG_CA3_inh = PopulationRateMonitor(G_DG_CA3_inh)
    popmon_CA1_inh = PopulationRateMonitor(G_CA1_inh)
    popmon_Sub_inh = PopulationRateMonitor(G_Sub_inh)
    popmon_EC_inh = PopulationRateMonitor(G_EC_inh)

    # Setting up a monitor that monitors the voltage of the LFP group.
    Mlfp = StateMonitor(G_lfp, 'v', record=True)

    # Setting up monitors that monitor the membrane potential, the noise current (and the stimulus current) for the selected excitatory/inhibitory neurons.
    statemon_DG_CA3_exc = StateMonitor(G_DG_CA3_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_CA1_exc = StateMonitor(G_CA1_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_Sub_exc = StateMonitor(G_Sub_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_EC_exc = StateMonitor(G_EC_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_DG_CA3_inh = StateMonitor(G_DG_CA3_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_CA1_inh = StateMonitor(G_CA1_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_Sub_inh = StateMonitor(G_Sub_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_EC_inh = StateMonitor(G_EC_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    monitors = [popmon_DG_CA3_exc, popmon_CA1_exc, popmon_Sub_exc, popmon_EC_exc, popmon_DG_CA3_inh, popmon_CA1_inh, popmon_Sub_inh, popmon_EC_inh, Mlfp, statemon_DG_CA3_exc, statemon_CA1_exc, statemon_Sub_exc, statemon_EC_exc, statemon_DG_CA3_inh, statemon_CA1_inh, statemon_Sub_inh, statemon_EC_inh]


    ############################################
    ########  Connecting Neuron Groups  ########
    ############################################
    # Configuring synaptic connections between neuron groups where in the function call 'Synapses()' the first parameter is the pre-synaptic group and the second parameter is the post-synaptic group. 
    # The third parameter 'on_pre' specifies the action to be taken when a pre-synaptic neuron fires where the synaptic event will increase the post-synaptic event by a certain amount defined by:
    # - gain => A scaling factor for the synaptic strength.
    # - g_max/siemens => The maximum conductance for the type of synapse normalized by dividing by 'siemens' which is a unit of conductance.
    # - glu_pre => The amount of glutamate released from the pre-synaptic neuron upon firing.
    # MIND: This does not create synapses but instead only specifies their dynamics. The actual synapse connections are created by using the function 'connect()'.
    synapses = []
    
    ################################      
    ## Connections Within Regions ##
    ################################
    # Generating within the DG_CA3 region the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_DG_CA3_e2e = Synapses(G_DG_CA3_exc, G_DG_CA3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_DG_CA3_e2e')
    S_DG_CA3_e2i = Synapses(G_DG_CA3_exc, G_DG_CA3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_DG_CA3_e2i')
    S_DG_CA3_i2e = Synapses(G_DG_CA3_inh, G_DG_CA3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_DG_CA3_i2e')
    S_DG_CA3_e2e.connect(p=f'{probabilities_within_regions['DG_CA3_e2e']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_DG_CA3_e2i.connect(p=f'{probabilities_within_regions['DG_CA3_e2i']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_DG_CA3_i2e.connect(p=f'{probabilities_within_regions['DG_CA3_i2e']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_DG_CA3_e2e)
    synapses.append(S_DG_CA3_e2i)
    synapses.append(S_DG_CA3_i2e)

    # Generating within the CA1 region the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_CA1_e2i = Synapses(G_CA1_exc, G_CA1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_CA1_e2i')
    S_CA1_i2e = Synapses(G_CA1_inh, G_CA1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_CA1_i2e')
    S_CA1_i2i = Synapses(G_CA1_inh, G_CA1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_CA1_i2i')
    S_CA1_e2i.connect(p=f'{probabilities_within_regions['CA1_e2i']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_CA1_i2e.connect(p=f'{probabilities_within_regions['CA1_i2e']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_CA1_i2i.connect(p=f'{probabilities_within_regions['CA1_i2i']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_CA1_e2i)
    synapses.append(S_CA1_i2e)
    synapses.append(S_CA1_i2i)

    # Generating within the Sub region the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_Sub_e2e = Synapses(G_Sub_exc, G_Sub_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_Sub_e2e')
    S_Sub_e2i = Synapses(G_Sub_exc, G_Sub_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_Sub_e2i')
    S_Sub_i2e = Synapses(G_Sub_inh, G_Sub_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_Sub_i2e')
    S_Sub_i2i = Synapses(G_Sub_inh, G_Sub_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_Sub_i2i')
    S_Sub_e2e.connect(p=f'{probabilities_within_regions['Sub_e2e']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_Sub_e2i.connect(p=f'{probabilities_within_regions['Sub_e2i']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_Sub_i2e.connect(p=f'{probabilities_within_regions['Sub_i2e']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_Sub_i2i.connect(p=f'{probabilities_within_regions['Sub_i2i']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_Sub_e2e)
    synapses.append(S_Sub_e2i)
    synapses.append(S_Sub_i2e)
    synapses.append(S_Sub_i2i)

    # Generating within the EC region the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_EC_e2i = Synapses(G_EC_exc, G_EC_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_within_EC_e2i')
    S_EC_i2e = Synapses(G_EC_inh, G_EC_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_within_EC_i2e')
    S_EC_e2i.connect(p=f'{probabilities_within_regions['EC_e2i']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_EC_i2e.connect(p=f'{probabilities_within_regions['EC_i2e']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_EC_e2i)
    synapses.append(S_EC_i2e)

    

    #################################      
    ## Connections Between Regions ##
    #################################
    # Generating from the DG_CA3 to the CA1 region the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_DG_CA3_CA1_e2e = Synapses(G_DG_CA3_exc, G_CA1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_DG_CA3_CA1_e2e')
    S_DG_CA3_CA1_e2i = Synapses(G_DG_CA3_exc, G_CA1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_DG_CA3_CA1_e2i')
    S_DG_CA3_CA1_i2e = Synapses(G_DG_CA3_inh, G_CA1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_DG_CA3_CA1_i2e')
    S_DG_CA3_CA1_i2i = Synapses(G_DG_CA3_inh, G_CA1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_DG_CA3_CA1_i2i')
    S_DG_CA3_CA1_e2e.connect(p=f'{probabilities_between_regions['DG_CA3_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_DG_CA3_CA1_e2i.connect(p=f'{probabilities_between_regions['DG_CA3_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_DG_CA3_CA1_i2e.connect(p=f'{probabilities_between_regions['DG_CA3_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_DG_CA3_CA1_i2i.connect(p=f'{probabilities_between_regions['DG_CA3_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_DG_CA3_CA1_e2e)
    synapses.append(S_DG_CA3_CA1_e2i)
    synapses.append(S_DG_CA3_CA1_i2e)
    synapses.append(S_DG_CA3_CA1_i2i)


    # Generating from the CA1 to the Sub region the connections between the groups of neurons which is a probabilistic process indicating that not all neurons are connected with each other but instead it is decided based on the provided probability and a distance measure.
    S_CA1_Sub_e2e = Synapses(G_CA1_exc, G_Sub_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_CA1_Sub_e2e')
    S_CA1_Sub_e2i = Synapses(G_CA1_exc, G_Sub_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_CA1_Sub_e2i')
    S_CA1_Sub_i2e = Synapses(G_CA1_inh, G_Sub_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_CA1_Sub_i2e')
    S_CA1_Sub_i2i = Synapses(G_CA1_inh, G_Sub_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_CA1_Sub_i2i')
    S_CA1_Sub_e2e.connect(p=f'{probabilities_between_regions['CA1_to_Sub']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_CA1_Sub_e2i.connect(p=f'{probabilities_between_regions['CA1_to_Sub']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_CA1_Sub_i2e.connect(p=f'{probabilities_between_regions['CA1_to_Sub']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_CA1_Sub_i2i.connect(p=f'{probabilities_between_regions['CA1_to_Sub']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    synapses.append(S_CA1_Sub_e2e)
    synapses.append(S_CA1_Sub_e2i)
    synapses.append(S_CA1_Sub_i2e)
    synapses.append(S_CA1_Sub_i2i)

    
    # Since we want the total number of connections from the CA1 to the EC region to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_CA1_EC_e2e_no_dist = Synapses(G_CA1_exc, G_EC_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_CA1_EC_e2e_dist = Synapses(G_CA1_exc, G_EC_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_CA1_EC_e2e_no_dist.connect(p=probabilities_between_regions['CA1_to_EC'])
    testing_S_CA1_EC_e2e_dist.connect(p=f'{probabilities_between_regions['CA1_to_EC']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_CA1_EC_e2e = len(testing_S_CA1_EC_e2e_dist.N_outgoing_pre) / len(testing_S_CA1_EC_e2e_no_dist.N_outgoing_pre)
    
    testing_S_CA1_EC_e2i_no_dist = Synapses(G_CA1_exc, G_EC_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_CA1_EC_e2i_dist = Synapses(G_CA1_exc, G_EC_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_CA1_EC_e2i_no_dist.connect(p=probabilities_between_regions['CA1_to_EC'])
    testing_S_CA1_EC_e2i_dist.connect(p=f'{probabilities_between_regions['CA1_to_EC']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_CA1_EC_e2i = len(testing_S_CA1_EC_e2i_dist.N_outgoing_pre) / len(testing_S_CA1_EC_e2i_no_dist.N_outgoing_pre)

    testing_S_CA1_EC_i2e_no_dist = Synapses(G_CA1_inh, G_EC_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_CA1_EC_i2e_dist = Synapses(G_CA1_inh, G_EC_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_CA1_EC_i2e_no_dist.connect(p=probabilities_between_regions['CA1_to_EC'])
    testing_S_CA1_EC_i2e_dist.connect(p=f'{probabilities_between_regions['CA1_to_EC']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_CA1_EC_i2e = len(testing_S_CA1_EC_i2e_dist.N_outgoing_pre) / len(testing_S_CA1_EC_i2e_no_dist.N_outgoing_pre)

    testing_S_CA1_EC_i2i_no_dist = Synapses(G_CA1_inh, G_EC_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_CA1_EC_i2i_dist = Synapses(G_CA1_inh, G_EC_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_CA1_EC_i2i_no_dist.connect(p=probabilities_between_regions['CA1_to_EC'])
    testing_S_CA1_EC_i2i_dist.connect(p=f'{probabilities_between_regions['CA1_to_EC']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_CA1_EC_i2i = len(testing_S_CA1_EC_i2i_dist.N_outgoing_pre) / len(testing_S_CA1_EC_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_CA1_EC_e2e = Synapses(G_CA1_exc, G_EC_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_CA1_EC_e2e')
    S_CA1_EC_e2i = Synapses(G_CA1_exc, G_EC_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_CA1_EC_e2i')
    S_CA1_EC_i2e = Synapses(G_CA1_inh, G_EC_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_CA1_EC_i2e')
    S_CA1_EC_i2i = Synapses(G_CA1_inh, G_EC_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_CA1_EC_i2i')
    S_CA1_EC_e2e.connect(p=probabilities_between_regions['CA1_to_EC']*ratio_CA1_EC_e2e)
    S_CA1_EC_e2i.connect(p=probabilities_between_regions['CA1_to_EC']*ratio_CA1_EC_e2i)
    S_CA1_EC_i2e.connect(p=probabilities_between_regions['CA1_to_EC']*ratio_CA1_EC_i2e)
    S_CA1_EC_i2i.connect(p=probabilities_between_regions['CA1_to_EC']*ratio_CA1_EC_i2i)
    synapses.append(S_CA1_EC_e2e)
    synapses.append(S_CA1_EC_e2i)
    synapses.append(S_CA1_EC_i2e)
    synapses.append(S_CA1_EC_i2i)

    
    # Since we want the total number of connections from the EC to the DG_CA3 region to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_EC_DG_CA3_e2e_no_dist = Synapses(G_EC_exc, G_DG_CA3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_DG_CA3_e2e_dist = Synapses(G_EC_exc, G_DG_CA3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_DG_CA3_e2e_no_dist.connect(p=probabilities_between_regions['EC_to_DG_CA3'])
    testing_S_EC_DG_CA3_e2e_dist.connect(p=f'{probabilities_between_regions['EC_to_DG_CA3']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_DG_CA3_e2e = len(testing_S_EC_DG_CA3_e2e_dist.N_outgoing_pre) / len(testing_S_EC_DG_CA3_e2e_no_dist.N_outgoing_pre)
    
    testing_S_EC_DG_CA3_e2i_no_dist = Synapses(G_EC_exc, G_DG_CA3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_DG_CA3_e2i_dist = Synapses(G_EC_exc, G_DG_CA3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_DG_CA3_e2i_no_dist.connect(p=probabilities_between_regions['EC_to_DG_CA3'])
    testing_S_EC_DG_CA3_e2i_dist.connect(p=f'{probabilities_between_regions['EC_to_DG_CA3']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_DG_CA3_e2i = len(testing_S_EC_DG_CA3_e2i_dist.N_outgoing_pre) / len(testing_S_EC_DG_CA3_e2i_no_dist.N_outgoing_pre)

    testing_S_EC_DG_CA3_i2e_no_dist = Synapses(G_EC_inh, G_DG_CA3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_DG_CA3_i2e_dist = Synapses(G_EC_inh, G_DG_CA3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_DG_CA3_i2e_no_dist.connect(p=probabilities_between_regions['EC_to_DG_CA3'])
    testing_S_EC_DG_CA3_i2e_dist.connect(p=f'{probabilities_between_regions['EC_to_DG_CA3']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_DG_CA3_i2e = len(testing_S_EC_DG_CA3_i2e_dist.N_outgoing_pre) / len(testing_S_EC_DG_CA3_i2e_no_dist.N_outgoing_pre)

    testing_S_EC_DG_CA3_i2i_no_dist = Synapses(G_EC_inh, G_DG_CA3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_DG_CA3_i2i_dist = Synapses(G_EC_inh, G_DG_CA3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_DG_CA3_i2i_no_dist.connect(p=probabilities_between_regions['EC_to_DG_CA3'])
    testing_S_EC_DG_CA3_i2i_dist.connect(p=f'{probabilities_between_regions['EC_to_DG_CA3']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_DG_CA3_i2i = len(testing_S_EC_DG_CA3_i2i_dist.N_outgoing_pre) / len(testing_S_EC_DG_CA3_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_EC_DG_CA3_e2e = Synapses(G_EC_exc, G_DG_CA3_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_EC_DG_CA3_e2e')
    S_EC_DG_CA3_e2i = Synapses(G_EC_exc, G_DG_CA3_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_EC_DG_CA3_e2i')
    S_EC_DG_CA3_i2e = Synapses(G_EC_inh, G_DG_CA3_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_EC_DG_CA3_i2e')
    S_EC_DG_CA3_i2i = Synapses(G_EC_inh, G_DG_CA3_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_EC_DG_CA3_i2i')
    S_EC_DG_CA3_e2e.connect(p=probabilities_between_regions['EC_to_DG_CA3']*ratio_EC_DG_CA3_e2e)
    S_EC_DG_CA3_e2i.connect(p=probabilities_between_regions['EC_to_DG_CA3']*ratio_EC_DG_CA3_e2i)
    S_EC_DG_CA3_i2e.connect(p=probabilities_between_regions['EC_to_DG_CA3']*ratio_EC_DG_CA3_i2e)
    S_EC_DG_CA3_i2i.connect(p=probabilities_between_regions['EC_to_DG_CA3']*ratio_EC_DG_CA3_i2i)
    synapses.append(S_EC_DG_CA3_e2e)
    synapses.append(S_EC_DG_CA3_e2i)
    synapses.append(S_EC_DG_CA3_i2e)
    synapses.append(S_EC_DG_CA3_i2i)

    
    # Since we want the total number of connections from the EC to the CA1 region to be equal when only using the given probability versus when using both the given probability and distance measure, we can examine how many connections are made for each case.
    # This is done separately for all four possible combinations of neuron type connections.
    testing_S_EC_CA1_e2e_no_dist = Synapses(G_EC_exc, G_CA1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_CA1_e2e_dist = Synapses(G_EC_exc, G_CA1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_CA1_e2e_no_dist.connect(p=probabilities_between_regions['EC_to_CA1'])
    testing_S_EC_CA1_e2e_dist.connect(p=f'{probabilities_between_regions['EC_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_CA1_e2e = len(testing_S_EC_CA1_e2e_dist.N_outgoing_pre) / len(testing_S_EC_CA1_e2e_no_dist.N_outgoing_pre)
    
    testing_S_EC_CA1_e2i_no_dist = Synapses(G_EC_exc, G_CA1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_CA1_e2i_dist = Synapses(G_EC_exc, G_CA1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    testing_S_EC_CA1_e2i_no_dist.connect(p=probabilities_between_regions['EC_to_CA1'])
    testing_S_EC_CA1_e2i_dist.connect(p=f'{probabilities_between_regions['EC_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_CA1_e2i = len(testing_S_EC_CA1_e2i_dist.N_outgoing_pre) / len(testing_S_EC_CA1_e2i_no_dist.N_outgoing_pre)

    testing_S_EC_CA1_i2e_no_dist = Synapses(G_EC_inh, G_CA1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_CA1_i2e_dist = Synapses(G_EC_inh, G_CA1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_CA1_i2e_no_dist.connect(p=probabilities_between_regions['EC_to_CA1'])
    testing_S_EC_CA1_i2e_dist.connect(p=f'{probabilities_between_regions['EC_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_CA1_i2e = len(testing_S_EC_CA1_i2e_dist.N_outgoing_pre) / len(testing_S_EC_CA1_i2e_no_dist.N_outgoing_pre)

    testing_S_EC_CA1_i2i_no_dist = Synapses(G_EC_inh, G_CA1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_CA1_i2i_dist = Synapses(G_EC_inh, G_CA1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens")
    testing_S_EC_CA1_i2i_no_dist.connect(p=probabilities_between_regions['EC_to_CA1'])
    testing_S_EC_CA1_i2i_dist.connect(p=f'{probabilities_between_regions['EC_to_CA1']}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    ratio_EC_CA1_i2i = len(testing_S_EC_CA1_i2i_dist.N_outgoing_pre) / len(testing_S_EC_CA1_i2i_no_dist.N_outgoing_pre)

    # These ratios can be used to multiply with the given probability in the 'connect()' function to ensure that the same number of connections would have been made if the distance measure was also included in the probability.
    S_EC_CA1_e2e = Synapses(G_EC_exc, G_CA1_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_EC_CA1_e2e')
    S_EC_CA1_e2i = Synapses(G_EC_exc, G_CA1_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_between_EC_CA1_e2i')
    S_EC_CA1_i2e = Synapses(G_EC_inh, G_CA1_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_EC_CA1_i2e')
    S_EC_CA1_i2i = Synapses(G_EC_inh, G_CA1_inh, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_between_EC_CA1_i2i')
    S_EC_CA1_e2e.connect(p=probabilities_between_regions['EC_to_CA1']*ratio_EC_CA1_e2e)
    S_EC_CA1_e2i.connect(p=probabilities_between_regions['EC_to_CA1']*ratio_EC_CA1_e2i)
    S_EC_CA1_i2e.connect(p=probabilities_between_regions['EC_to_CA1']*ratio_EC_CA1_i2e)
    S_EC_CA1_i2i.connect(p=probabilities_between_regions['EC_to_CA1']*ratio_EC_CA1_i2i)
    synapses.append(S_EC_CA1_e2e)
    synapses.append(S_EC_CA1_e2i)
    synapses.append(S_EC_CA1_i2e)
    synapses.append(S_EC_CA1_i2i)

    
    ########################################################      
    ## Connections Between Electrode and Electrode Region ##
    ########################################################
    # Configuring synaptic connections between the excitatory neurons and the LFP electrode based on what the region of the LFP electrode is.
    if current_variables['LFP_electrode_region'] == "DG_CA3":
        S_lfp = Synapses(G_DG_CA3_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                               v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_region'] == "CA1":
        S_lfp = Synapses(G_CA1_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_region'] == "Sub":
        S_lfp = Synapses(G_Sub_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    if current_variables['LFP_electrode_region'] == "EC":
        S_lfp = Synapses(G_EC_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')

    # Ensuring LFP voltage is updated after neuron groups.
    S_lfp.summed_updaters['v_post'].when = 'after_groups' 

    # Generating the connections between the excitatory neurons and the LFP electrode. Here, all of them will be connected since there is no probability defined.
    S_lfp.connect()

    # Setting the weight for LFP calculation which is scaled by the Euclidean distance measure.
    S_lfp.w = '(29e3 * umetre ** 2)/(4*pi*sigma)/((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)**.5'

    synapses.append(S_lfp)


    ############################################
    ########  Final Network Definition  ########
    ############################################
    net = Network(neuron_groups, synapses, monitors)

    
    return net, synapses, monitors

<br></br><br></br>

<h2 style="font-size: 40px;">Simulation Functions</h2>

In [17]:
# Runs the simulation of the neural network in discrete time fragments which includes a mechanism to alter neuron parameters based on a firing rate threshold, which simulates a treatment effect.
def run_granular_simulation(net, variables, treatment_settings, monitors):
    
    print('#######################')
    print('# Starting Simulation #')
    print('#######################')
    print()

    # Extracting the passed on parameters.
    total_duration = variables['duration']
    input_signal = read_input_signal(variables['input_signal_file'])
    time_fragment, firing_rate_threshold = treatment_settings
    popmon_DG_CA3_exc, popmon_CA1_exc, popmon_Sub_exc, popmon_EC_exc, popmon_DG_CA3_inh, popmon_CA1_inh, popmon_Sub_inh, popmon_EC_inh, Mlfp, statemon_DG_CA3_exc, statemon_CA1_exc, statemon_Sub_exc, statemon_EC_exc, statemon_DG_CA3_inh, statemon_CA1_inh, statemon_Sub_inh, statemon_EC_inh = monitors
    
    # Setting the potassium equilibrium potentials for both the excitatory and inhibitory neurons.
    Eke = variables['Eke_baseline']
    Eki = variables['Eki_baseline']
    Eke_baseline = variables['Eke_baseline']
    Eki_baseline = variables['Eki_baseline']
    
    print('Treatment Parameters:', 'time sensitivity', time_fragment, 'FR Threshold:', firing_rate_threshold)
    print()

    # Converting the time fragments to milliseconds and determining the number of batches.
    time_fragment_ms = int(time_fragment/ms)
    num_batches = int(total_duration / time_fragment)

    # Setting the population monitor of which the average firing rate will be evaluated to be the one corresponding to the topology the stimulus is in.
    if variables['stimulus_regions'][0] == "DG_CA3":
        population_monitor = popmon_DG_CA3_exc
    elif variables['stimulus_regions'][0] == "CA1":
        population_monitor = popmon_CA1_exc

    # For every batch, we run the simulation on the network 'net'. Here the 'tqdm()' function is used which creates a progress bar.
    for i in tqdm(range(num_batches), desc="Running Simulation"): 
        net.run(time_fragment)

        # If after running a time fragment, the average firing rate of the excitatory neurons over the last time fragment exceeds the threshold, treatment is initiated.
        # This statement refers to the scenario where epilepsy is detected and treated by adjusting the potassium equilibrium potentials of the excitatory and inhibitory neurons.
        if np.mean(population_monitor.rate[-time_fragment_ms:]) > firing_rate_threshold:
            Eke = variables['Eke_treatment']
            Eki = variables['Eki_treatment']

<br></br>

In [18]:
# This function manages the overall process of setting up and running multiple instances of a neural network simulation as after setting everything up, the function 'run_granular_simulation()' is called.
# In addition, it also handles the storing of the results.
def run_model_loop(variables):

    # Checking whether lengths of each variable list is equal.
    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')
    
    # For every run provided in the 'run_id' field of the variables dictionary we perform the simulation (which requires some setting up first).
    for i in range(len(variables['run_id'])):
        
        ##########################################
        ########  Basic Simulation Setup  ########
        ##########################################
        # Resetting the state of the simulation environment (to avoid interference from the previous run) and seeting the default simulation time step.
        start_scope()
        defaultclock.dt = 0.001*second   

        # Extracting the list of variables required for the current run.
        current_variables = {key: variables[key][i] for key in variables}
        print(current_variables['noise_exc'])

        # Retrieving the treatment settings.
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]

        # Creating a folder to store the results.
        run_id = current_variables['run_id']
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)

        
        #######################################
        ########  Creating Topologies  ########
        #######################################
        # Creating the topologies of both the excitatory and inhibitory neurons.
        region_data, region_geometrics, excitatory_positions, inhibitory_positions = create_complete_neuron_topology_hippocampus(current_variables['N'], current_variables['region_densities'], current_variables['region_volumes'], current_variables['excitatory_ratios'], current_variables['region_names'])

        print(region_geometrics)
        
        # Assigning the topologies of the excitatory neurons of the different regions to separate variables.
        topology_DG_CA3_exc = excitatory_positions['DG_CA3']
        topology_CA1_exc = excitatory_positions['CA1']
        topology_Sub_exc = excitatory_positions['Sub']
        topology_EC_exc = excitatory_positions['EC']

        # Assigning the topologies of the inhibitory neurons of the different regions to separate variables.
        topology_DG_CA3_inh = inhibitory_positions['DG_CA3']
        topology_CA1_inh = inhibitory_positions['CA1']
        topology_Sub_inh = inhibitory_positions['Sub']
        topology_EC_inh = inhibitory_positions['EC']

        # Calculating the lengths of the excitatory and inhibitory topologies.
        excitatory_topologies_lengths = [len(topology_DG_CA3_exc[0]), len(topology_CA1_exc[0]), len(topology_Sub_exc[0]), len(topology_EC_exc[0])]
        inhibitory_topologies_lengths = [len(topology_DG_CA3_inh[0]), len(topology_CA1_inh[0]), len(topology_Sub_inh[0]), len(topology_EC_inh[0])]

        # Adding all the topologies of the neurons to a list.
        topologies = [topology_DG_CA3_exc, topology_CA1_exc, topology_Sub_exc, topology_EC_exc, topology_DG_CA3_inh, topology_CA1_inh, topology_Sub_inh, topology_EC_inh] 
        topology_names = ['DG_CA3_exc', 'CA1_exc', 'Sub_exc', 'EC_exc', 'DG_CA3_inh', 'CA1_inh', 'Sub_inh', 'EC_inh']

        # Updating the 'current_variables' by adding the 'region_data' and 'region_geometrics'.
        current_variables['region_data'] = region_data
        current_variables['region_geometrics'] = region_geometrics

        # Plotting all the hippocampal regions/topologies in a single 3D plot.
        plot_all_hippocampal_topologies(topologies, topology_names, run_id)


        ############################################
        ########  Setting up Stimulus Mask  ########
        ############################################
        # Defining which topologies should be considered for the creation of the stimulus mask according to the stimulus regions.
        topology_exc_stimulus = [[],[],[]]
        for stimulus_region in current_variables['stimulus_regions']:
            for i, (sublist1, sublist2) in enumerate(zip_longest(topology_exc_stimulus, excitatory_positions[stimulus_region], fillvalue=[])):
                topology_exc_stimulus[i].extend(sublist2)

        # If the shape of the stimulus mask is set to 'all', then there is no need to check the mask first.
        if current_variables['shape_stimulus_mask'] == "all":
            
            ##########################################
            ########  Creating Stimulus Mask  ########
            ##########################################
            stimulus_mask_exc = create_all_mask(topology_exc_stimulus)
        
        elif current_variables['shape_stimulus_mask'] == "perc_of_all":

            ##########################################
            ########  Creating Stimulus Mask  ########
            ##########################################
            stimulus_mask_exc = create_prec_of_all_mask(topology_exc_stimulus, current_variables['stimulus_mask_all_perc'])
            
        else:
            # Checking whether the stimulus mask with the provided coordinate center and radius can be applied to the list of regions by calling the function 'checking_mask()'.
            coord_of_stimulus, stimulus_radius_or_edge_length = checking_mask(current_variables['shape_stimulus_mask'], current_variables['stimulus_regions'], current_variables['stimulus_center_coordinates'], current_variables['stimulus_radius_or_edge_length'], current_variables['region_names'], region_geometrics)
            
            # Setting the overall geometry settings of the stimulus mask which includes both the middle point as well as the radius.
            stimulus_geometry_settings = [coord_of_stimulus, stimulus_radius_or_edge_length]
            
            ##########################################
            ########  Creating Stimulus Mask  ########
            ##########################################
            # Creating the stimulus mask for the defined 'topology_exc_stimulus' with as shape the 'current_variables['shape_stimulus_mask']'.
            if current_variables['shape_stimulus_mask'] == "spherical":
                stimulus_mask_exc = create_spherical_mask(topology_exc_stimulus, stimulus_geometry_settings)
            elif current_variables['shape_stimulus_mask'] == "cubical":
                stimulus_mask_exc = create_cubical_mask(topology_exc_stimulus, stimulus_geometry_settings)

       
        ############################################
        ########  Setting up Electrode  ############
        ############################################
        # Adding the coordinates of the electrode to the 'variables' and 'current_variables'.
        # variables['coord_of_electrode'] = populate_electrode_positions(variables)
        # current_variables['coord_of_electrode'] = populate_electrode_positions(current_variables)


        #############################################
        ########  Setting up Treatment Mask  ########
        #############################################
        # Defining which topologies should be considered for the creation of the treatment mask according to the treatment regions.
        topology_exc_treatment = [[],[],[]]
        topology_inh_treatment = [[],[],[]]
        for treatment_region in current_variables['treatment_regions']:
            for i, (sublist1, sublist2) in enumerate(zip_longest(topology_exc_treatment, excitatory_positions[treatment_region], fillvalue=[])):
                topology_exc_treatment[i].extend(sublist2)
            for i, (sublist1, sublist2) in enumerate(zip_longest(topology_inh_treatment, inhibitory_positions[treatment_region], fillvalue=[])):
                topology_inh_treatment[i].extend(sublist2)

        # If the shape of the treatment mask is set to 'all', then there is no need to check the mask first.
        if current_variables['shape_treatment_mask'] == "all":

            ###########################################
            ########  Creating Treatment Mask  ########
            ###########################################
            treatment_mask_exc = create_all_mask(topology_exc_treatment)
            treatment_mask_inh = create_all_mask(topology_inh_treatment)

        elif current_variables['shape_treatment_mask'] == "perc_of_all":

            ###########################################
            ########  Creating Treatment Mask  ########
            ###########################################
            treatment_mask_exc = create_prec_of_all_mask(topology_exc_treatment, current_variables['treatment_mask_all_perc'])
            treatment_mask_inh = create_prec_of_all_mask(topology_inh_treatment, current_variables['treatment_mask_all_perc'])
            
        else:
            # Checking whether the treatment mask with the provided coordinate center and radius can be applied to the list of regions by calling the function 'checking_treatment_mask()'.
            coord_of_treatment, treatment_radius_or_edge_length = checking_mask(current_variables['shape_treatment_mask'], current_variables['treatment_regions'], current_variables['treatment_center_coordinates'], current_variables['treatment_radius_or_edge_length'], current_variables['region_names'], region_geometrics)
            
            # Setting the overall geometry settings of the treatment mask which includes both the middle point as well as the radius.
            treatment_geometry_settings = [coord_of_treatment, treatment_radius_or_edge_length]
            
            ###########################################
            ########  Creating Treatment Mask  ########
            ###########################################
            # Creating the treatment mask for the defined 'topology_exc_treatment' with as shape the 'current_variables['shape_treatment_mask']'.
            if current_variables['shape_treatment_mask'] == "spherical":
                treatment_mask_exc = create_spherical_mask(topology_exc_treatment, treatment_geometry_settings)
                treatment_mask_inh = create_spherical_mask(topology_inh_treatment, treatment_geometry_settings)
            elif current_variables['shape_treatment_mask'] == "cubical":
                treatment_mask_exc = create_cubical_mask(topology_exc_treatment, treatment_geometry_settings)
                treatment_mask_inh = create_cubical_mask(topology_inh_treatment, treatment_geometry_settings)

        treatment_masks = [treatment_mask_exc, treatment_mask_inh]

        # Plotting the stimulus and treatment masks in a 3D plot which also features all hippocampal topologies.
        plotting_masks(topologies, topology_names, [current_variables['topology_names_stimulus'], current_variables['topology_names_treatment']], [stimulus_mask_exc, treatment_masks], excitatory_topologies_lengths, inhibitory_topologies_lengths, run_id)


    
    
        ########################################
        ########  Initializing Network  ########
        ########################################
        # Instantiating the network and setting up the monitors.
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_DG_CA3_exc, popmon_CA1_exc, popmon_Sub_exc, popmon_EC_exc, popmon_DG_CA3_inh, popmon_CA1_inh, popmon_Sub_inh, popmon_EC_inh, Mlfp, statemon_DG_CA3_exc, statemon_CA1_exc, statemon_Sub_exc, statemon_EC_exc, statemon_DG_CA3_inh, statemon_CA1_inh, statemon_Sub_inh, statemon_EC_inh = monitors
        
        # Writing the network statistics to a file.
        #write_network_statistics(synapses, current_variables['N'], run_id)

        # Running the simulation with the dynamic objects created above.
        run_granular_simulation(net, current_variables, treatment_settings, monitors)


        ###########################################
        ########  Saving Firing Rate Data  ########
        ###########################################
        exc_color = 'blue'  # Color for excitatory neurons
        inh_color = 'orange'   # Color for inhibitory neurons

        # Saving the firing rate data for every single region.
        np.savetxt(f'./results/{run_id}/firing_rate_DG_CA3_exc.txt', popmon_DG_CA3_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_DG_CA3_inh.txt', popmon_DG_CA3_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_CA1_exc.txt', popmon_CA1_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_CA1_inh.txt', popmon_CA1_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_Sub_exc.txt', popmon_Sub_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_Sub_inh.txt', popmon_Sub_inh.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_EC_exc.txt', popmon_EC_exc.rate)
        np.savetxt(f'./results/{run_id}/firing_rate_EC_inh.txt', popmon_EC_inh.rate)

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the DG_CA3 region.
        plt.plot(popmon_DG_CA3_exc.t, popmon_DG_CA3_exc.rate, label='DG_CA3_exc', color=exc_color)
        plt.plot(popmon_DG_CA3_inh.t, popmon_DG_CA3_inh.rate, label='DG_CA3_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_DG_CA3_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_DG_CA3_exc.t, popmon_DG_CA3_exc.rate, label='DG_CA3_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_DG_CA3_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_DG_CA3_inh.t, popmon_DG_CA3_inh.rate, label='DG_CA3_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_DG_CA3_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the CA1 region.
        plt.plot(popmon_CA1_exc.t, popmon_CA1_exc.rate, label='CA1_exc', color=exc_color)
        plt.plot(popmon_CA1_inh.t, popmon_CA1_inh.rate, label='CA1_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_CA1_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_CA1_exc.t, popmon_CA1_exc.rate, label='CA1_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_CA1_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_CA1_inh.t, popmon_CA1_inh.rate, label='CA1_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_CA1_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the Sub region.
        plt.plot(popmon_Sub_exc.t, popmon_Sub_exc.rate, label='Sub_exc', color=exc_color)
        plt.plot(popmon_Sub_inh.t, popmon_Sub_inh.rate, label='Sub_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_Sub_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_Sub_exc.t, popmon_Sub_exc.rate, label='Sub_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_Sub_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_Sub_inh.t, popmon_Sub_inh.rate, label='Sub_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_Sub_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting the firing rate data for both the excitatory and inhibitory neurons of the EC region.
        plt.plot(popmon_EC_exc.t, popmon_EC_exc.rate, label='EC_exc', color=exc_color)
        plt.plot(popmon_EC_inh.t, popmon_EC_inh.rate, label='EC_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_EC_both.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_EC_exc.t, popmon_EC_exc.rate, label='EC_exc', color=exc_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_EC_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(popmon_EC_inh.t, popmon_EC_inh.rate, label='EC_inh', color=inh_color)
        plt.legend()
        plt.savefig(f'./results/{run_id}/firing_rate_EC_inh.png', bbox_inches='tight')
        plt.close()


        #####################################
        ########  Saving Noise Data  ########
        #####################################
        # Plotting and saving the noise for the excitatory and inhibitory neurons of the DG_CA3 region.
        plt.plot(statemon_DG_CA3_exc.t, statemon_DG_CA3_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_DG_CA3_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_DG_CA3_inh.t, statemon_DG_CA3_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_DG_CA3_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the CA1 region.
        plt.plot(statemon_CA1_exc.t, statemon_CA1_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_CA1_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_CA1_inh.t, statemon_CA1_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_CA1_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the Sub region.
        plt.plot(statemon_Sub_exc.t, statemon_Sub_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_Sub_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_Sub_inh.t, statemon_Sub_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_Sub_inh.png', bbox_inches='tight')
        plt.close()

        # Plotting and saving the noise for the excitatory and inhibitory neurons of the EC region.
        plt.plot(statemon_EC_exc.t, statemon_EC_exc.I_noise[4]/nA, label='exc')
        plt.savefig(f'./results/{run_id}/noise_EC_exc.png', bbox_inches='tight')
        plt.close()
        plt.plot(statemon_EC_inh.t, statemon_EC_inh.I_noise[4]/nA, label='inh')
        plt.savefig(f'./results/{run_id}/noise_EC_inh.png', bbox_inches='tight')
        plt.close()


        ###################################
        ########  Saving LFP Data  ########
        ###################################
        # Plotting and saving the recorded LFP voltage values in millivolts from the 'Mlfp' monitor. 
        np.savetxt(f'./results/{run_id}/voltage_LFP.txt', Mlfp.v[0]/mV)
        plot(Mlfp.t/ms, Mlfp.v[0]/mV)
        plt.savefig(f'./results/{run_id}/voltage_LFP.png', bbox_inches='tight')
        plt.close()

        
        time.sleep(1)

<br></br><br></br>

<h2 style="font-size: 40px;">Simulation Variables</h2>

In [19]:
########################################
########  Simulation Variables  ########
########################################

# Setting the number of times the simulation will be performed.
copy_times = 1


# Defining the volumes, neuron densities, and excitatory to inhibitory ratios of the regions.
region_volumes = {"DG_CA3": 128.85, 
                  "CA1": 177.5, 
                  "Sub": 91.55,
                  "EC": 88.25}
DG_neuron_density = 87100
CA3_neuron_density = 15100
region_densities_dict = {"DG_CA3": (DG_neuron_density + CA3_neuron_density) / 2,
                         "CA1": 7900, 
                         "Sub": 21500,
                         "EC": 27400}
region_excitatory_ratios_dict = {"DG_CA3": 0.8, 
                                 "CA1": 0.8, 
                                 "Sub": 0.8,
                                 "EC": 0.8}

# Defining the connection probabilities between neurons from different regions.
DG_proportion = DG_neuron_density / (DG_neuron_density + CA3_neuron_density)
CA3_proportion = CA3_neuron_density / (DG_neuron_density + CA3_neuron_density)
probabilities_between_regions = {"DG_CA3_to_CA1": (DG_proportion * 0) + (CA3_proportion * 0.1),
                                 "CA1_to_Sub": 0.55, 
                                 "CA1_to_EC": 0.1,
                                 "EC_to_DG_CA3": (DG_proportion * 0.1) + (CA3_proportion * 0.06),
                                 "EC_to_CA1": 0.06}

# Defining the connection probabilities between neurons from the same region.
probabilities_within_regions = {"DG_CA3_e2e": (DG_proportion * 0) + (CA3_proportion * 0.56),
                                "DG_CA3_e2i": (DG_proportion * 0.06) + (CA3_proportion * 0.75),
                                "DG_CA3_i2e": (DG_proportion * 0.14) + (CA3_proportion * 0.75),
                                "CA1_e2i": 0.28,
                                "CA1_i2e": 0.3,
                                "CA1_i2i": 0.7,
                                "Sub_e2e": 0.0523,
                                "Sub_e2i": 0.185,
                                "Sub_i2e": 0.405,
                                "Sub_i2i": 0.23,
                                "EC_e2i": 0.37,
                                "EC_i2e": 0.54}
                            
# Defining the vocabulary of variables.
variables = {

    # Defining basic run variables.
    "run_id": ['Results 1'],
    "duration": [4000*ms]*copy_times,
    "copy_times": [copy_times]*copy_times,

    # Defining the total number of neurons of the model.
    "N": [17000]*copy_times,

    # Defining the bounds used in the random topology model.
    "bounds": [[0.6, 0.6, 0.6]]*copy_times,
    
    # Defining the number of neurons per mm^3 and the excitatory ratio for each of the regions.
    "region_names": [['DG_CA3', 'CA1', 'Sub', 'EC']]*copy_times,
    "region_volumes": [dict(region_volumes)]*copy_times,
    "region_densities": [dict(region_densities_dict)]*copy_times,
    "excitatory_ratios": [dict(region_excitatory_ratios_dict)]*copy_times,

    # Defining the potassium equilibrium potential for both the excitatory and inhibitory neurons.
    # - Healthy mode: Eke_baseline = -90mV
    # - Epileptic mode: Eke_baseline = -84mV
    "Eke_baseline": [-84*mV]*copy_times, 
    "Eki_baseline": [-90*mV]*copy_times,

    # Defining the noise affecting the excitatory and inhibitory neurons.
    "noise_exc": [[0.07, 0.075]*nA]*copy_times, # OLD: [0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Defining the base probabilities of connections between neurons from different regions:
    # - p_e2e => Probability of an excitatory to excitatory neuron (synapse) connection.
    # - p_e2i => Probability of an excitatory to inhibitory neuron (synapse) connection.
    # - p_i2e => Probability of an inhibitory to excitatory neuron (synapse) connection.
    # - p_i2i => Probability of an inhibitory to inhibitory neuron (synapse) connection.
    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500.
    #"p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, 

    # Defining the connection probabilities between neurons from different and the same regions.
    "probabilities_between_regions": [dict(probabilities_between_regions)]*copy_times,
    "probabilities_within_regions": [dict(probabilities_within_regions)]*copy_times,

    # Defining from which file the stimulus originates.
    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, 

    ## Defining which parameters can be tweaked for the stimulus mask:
    # - stimulus regions: The regions that will feature the stimulus mask.
    # - stimulus center coordinates: The center coordinates of the stimulus mask.
    # - stimulus radius/edge length: The radius/edge length of the stimulus mask.
    # - stimulus shape: The shape of the stimulus mask.
    "stimulus_regions": [['DG_CA3']]*copy_times,
    "topology_names_stimulus": [['DG_CA3_exc']]*copy_times,
    "stimulus_center_coordinates": [[0.55,0.33,0.28]]*copy_times,
    "stimulus_radius_or_edge_length": [0.3]*copy_times,
    "shape_stimulus_mask": ['all']*copy_times, # 'spherical', 'cubical', 'all'
    "stimulus_mask_all_perc": [50]*copy_times, # used for the 'create_prec_of_all_mask()' function.
    
    ## Defining which parameters are needed for the creation of the treatment mask:
    # - treatment regions: The regions that will feature the treatment mask.
    # - stimulus center coordinates: The center coordinates of the stimulus mask.
    # - stimulus radius/edge length: The radius/edge length of the stimulus mask.
    # - treatment shape: The shape of the treatment mask.
    "treatment_regions": [['Sub']],
    "topology_names_treatment": [['Sub_exc', 'Sub_inh']],
    "treatment_center_coordinates": [[1.15, 0.94, 0.28]],
    "treatment_radius_or_edge_length": [0.13],
    "shape_treatment_mask": ['spherical'], # 'spherical', 'cubical', 'all', 'perc_of_all'
    "treatment_mask_all_perc": [50]*copy_times, # used for the 'create_prec_of_all_mask()' function.
    "distance_between_masks": [0]*copy_times, # the distance required between the stimulus and treatment mask.

    ## Defining other parameters that can be tweaked for treatment:
    # - firing rate threshold: Rate at which the treatment should be activated.
    # - device sensitivity: Frequency with which is checked if the firing rate is above the threshold.
    # - Eke treatment: The potassium equilibrium potential for the excitatory neurons once treatment is activated.
    # - Eke treatment: The potassium equilibrium potential for the inhibitory neurons once treatment is activated (unchanged from untreated).
    "firing_rate_threshold": [5*Hz]*copy_times,
    "device_sensitivity": [8*ms]*copy_times,
    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,

    # Defining which region should feature the LFP electrode.
    "LFP_electrode_region": ['DG_CA3']*copy_times
}

<br></br><br></br>

<h2 style="font-size: 40px;">Running the Simulation</h2>

In [ ]:
# Running the simulation with the variables.
run_model_loop(variables)